In [28]:
!pip install catboost


# Train Forex Models With Regularized Parameters

## Version 2
* Temporal features False (Month, Day of week, Week Of Month)
* Hour 4 Features Lag by 7 optional
* Day Features (RSI, RSI crossover, RSI-Based SMA, ) Lag by 9
* Starting Date June 2020 Training Till September 2023
* Ending Date May 2024
* Hour 4 Features 'relative range', 'candle type', 'heikin ashi' 
* Hyperparameters Regularized
* Removing CHF

* Evaluation Metric 
* Sharp Ratio 
* Accuracy by net gains

Version 1 Regularized On Two Months April May On Two Conditions :
 
 1- hyper parameters chosen should give positive gains for any currency pairs for those two months
 
 2- Previous months accuracy should be 50% and net gains accuracy close to 50% or 52%

Results :- (pips)
* ❌ December -3080
* ✅ January 4844
* ✅ February 1288
* ✅ March 1883
* ✅ April 
* ✅ May


📉 Not all models were in positive gains in previous month despite all performing good on april may with positive gains

⭐ But parameters performing good on two different months gave them ability to keep majority of currency pairs model in positive gain

💢 Successfully managed to make December as profitable even though not recommended for trading by trading community.





In [ ]:

import pandas as pd
import numpy as np

results_dict = {}
day_features, hour4_features = load_features_files()
forex_pairs = [
    'AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'AUDUSD',
    'CADCHF', 'CADJPY',
    'CHFJPY', 
    'EURAUD', 'EURCAD', 'EURCHF', 'EURGBP', 
    'EURJPY', 'EURNZD', 'EURUSD',
    'GBPAUD', 'GBPCAD', 'GBPCHF', 
    'GBPJPY', 'GBPUSD', 'GBPNZD',
    'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDUSD',  
    'USDCHF', 'USDCAD', 'USDJPY'
        ]

symbol_hyperparameter = [
 'AUDJPY', 'AUDUSD', 'AUDCAD', 'AUDCHF', 'AUDNZD',
 'CADJPY', 
 'EURAUD', 'EURCAD', 'EURUSD', 'EURGBP', 'EURNZD',
 'GBPCAD', 'GBPCHF', 'GBPUSD', 'GBPNZD',
 'NZDCHF', 'NZDJPY',
 'USDCHF', 'USDJPY', 'USDCAD',
 'CADCHF', 
 'NZDCAD', 'NZDUSD']

# symbols = ['USDCAD']

for symbol in forex_pairs:
    
    
    day_data, hour4_data = get_day_hour4_features(symbol, day_features, hour4_features)

    X_train_scaled, X_test_scaled, y_train, y_test = get_features_transformed(symbol, day_data, hour4_data)
    
    print(symbol)
    # model = load_model(symbol)
    
    if symbol in symbol_hyperparameter: 
      
       symbol_parameters = load_parameters(symbol)
       iteration, learning_rate, depth = symbol_parameters['Iterations'], symbol_parameters['Lr'], symbol_parameters['depth']
       parameters = [ iteration, learning_rate, depth ] 
       model = train_model(X_train_scaled, X_test_scaled, y_train, y_test, parameters=parameters)
    
    else:
      model = train_model(X_train_scaled, X_test_scaled, y_train, y_test)

    # model = load_model(symbol)
    gains , accuracy = evaluate_model_return_gains_accuracy(symbol, model, X_test_scaled,  y_test)
    
     # 2  # 1 --> May 2024 , # 2 --> April 2024
    step = 1 # ____          May 
    # step = 2 # ____       April
    cluster = 22 # possible days of trading in a year data ends at 31 May
    gains, accuracy = evaluate_model_return_gains_accuracy(symbol, model, X_test_scaled, y_test, custom_sample= [step, cluster] ) # custom_sample= None, last_cluster = None  
    net_gains = gains['net_gains']
    
    accuracy_by_net_gains = gains['accuracy_by_net_gains']
    results_dict[symbol] = { 'net_gains': int(net_gains), 'accuracy': int(accuracy), 'month': 'may', 
                             'accuracy_by_net_gains': int(accuracy_by_net_gains)  }
    save_model(symbol, model)
    

    # accuracy_by_net_gains = gains['accuracy_by_net_gains_before']
    accuracy_by_net_gains_before = gains['accuracy_by_net_gains_before']

    print('net profit ', net_gains)
    # print('accuracy_by_net_gains \n',accuracy_by_net_gains )
    print('\naccuracy by net gains on previous test data ', accuracy_by_net_gains_before)
    print("accuracy on all test points excluding last 22 points ", accuracy )
    




# Code Structure To See Model Accuracy For A Symbol 

In [29]:
day_features, hour4_features = load_features_files()


In [219]:

def test_model(symbol, iteration, learning_rate, depth, window_size=9, alpha_type = "gamma", save_model=False ):

    day_data, hour4_data = get_day_hour4_features(symbol, day_features, hour4_features)

    X_train_scaled, X_test_scaled, y_train, y_test, X_test = get_features_transformed(symbol, day_data, hour4_data,  X_test_re=True,  window_size=window_size, alpha_type=alpha_type ) 

    print(symbol)
    # model = load_model(symbol)


    # symbol_parameters = load_parameters(symbol)
    # iteration, learning_rate, depth = symbol_parameters['Iterations'], symbol_parameters['Lr'], symbol_parameters['depth']


    # iteration, learning_rate, depth = 15, 0.68, 6 # AUDNZD

    # iteration, learning_rate, depth = 210, 0.19, 7 # AUDCAD

    # iteration, learning_rate, depth = 17, 0.9, 7 # AUDCAD



    parameters = [ iteration, learning_rate, depth ]
    
    # model = finetune_model(X_train_scaled, X_test_scaled, y_train, y_test, parameters=parameters)
    model = train_model(X_train_scaled, X_test_scaled, y_train, y_test, parameters=parameters)
    if save_model == True : save_model_v2(symbol, model, parameters, alpha_type, window_size)
    # if save_model == True :     
    #     save_model(symbol, model)

        

    print("MAY")

    step = 1 # ____          May 
    # step = 2 # ____       April
    cluster = 22 # possible days of trading in a year data ends at 31 May
    gains, accuracy = evaluate_model_return_gains_accuracy(symbol, model, X_test_scaled, y_test, custom_sample= [step, cluster], X_test=X_test ) # custom_sample= None, last_cluster = None  
    net_gains = gains['net_gains']

    accuracy_by_net_gains = gains['accuracy_by_net_gains']
    accuracy_by_net_gains_before = gains['accuracy_by_net_gains_before']

    print('net profit ', net_gains)
    print('\naccuracy by net gains on previous test data ', accuracy_by_net_gains_before)
    print("accuracy on all test points excluding last 22 points ", accuracy )

    print("APRIL")
    step = 2 # ____       April
    cluster = 22
    # cluster = 22 # possible days of trading in a year data ends at 31 May
    gains, accuracy = evaluate_model_return_gains_accuracy(symbol, model, X_test_scaled, y_test, custom_sample= [step, cluster], X_test=X_test ) # custom_sample= None, last_cluster = None  
    net_gains = gains['net_gains']

    accuracy_by_net_gains = gains['accuracy_by_net_gains']

    accuracy_by_net_gains_before = gains['accuracy_by_net_gains_before']

    print('net profit ', net_gains)
    print('\naccuracy by net gains on previous test data ', accuracy_by_net_gains_before)
    print("accuracy on all test points excluding last 22 points ", accuracy )



    print("MARCH")
    step = 3 # ____      March
    cluster = 22 # possible days of trading in a year data ends at 31 May
    gains, accuracy = evaluate_model_return_gains_accuracy(symbol, model, X_test_scaled, y_test, custom_sample= [step, cluster], X_test=X_test ) # custom_sample= None, last_cluster = None  
    net_gains = gains['net_gains']

    accuracy_by_net_gains = gains['accuracy_by_net_gains']

    accuracy_by_net_gains_before = gains['accuracy_by_net_gains_before']

    print('net profit ', net_gains)
    print('\naccuracy by net gains on previous test data ', accuracy_by_net_gains_before)
    print("accuracy on all test points excluding last 22 points ", accuracy )
    
    print("Iterations: ",iteration)
    print("Learning rate: ",learning_rate) 
    print("Depth: ",depth)
    
    
    print("FEB")
    step = 4 # ____      Feb
    cluster = 22 # possible days of trading in a year data ends at 31 May
    gains, accuracy = evaluate_model_return_gains_accuracy(symbol, model, X_test_scaled, y_test, custom_sample= [step, cluster], X_test=X_test ) # custom_sample= None, last_cluster = None  
    net_gains = gains['net_gains']

    accuracy_by_net_gains = gains['accuracy_by_net_gains']
    accuracy_by_net_gains_before = gains['accuracy_by_net_gains_before']

    print('net profit ', net_gains)
    print('\naccuracy by net gains on previous test data ', accuracy_by_net_gains_before)
    print("accuracy on all test points excluding last 22 points ", accuracy )
    
    print("JAN")
    step = 5 # ____      Jan
    cluster = 22 # possible days of trading in a year data ends at 31 May
    gains, accuracy = evaluate_model_return_gains_accuracy(symbol, model, X_test_scaled, y_test, custom_sample= [step, cluster], X_test=X_test ) # custom_sample= None, last_cluster = None  
    net_gains = gains['net_gains']

    accuracy_by_net_gains = gains['accuracy_by_net_gains']
    accuracy_by_net_gains_before = gains['accuracy_by_net_gains_before']

    print('net profit ', net_gains)
    print('\naccuracy by net gains on previous test data ', accuracy_by_net_gains_before)
    print("accuracy on all test points excluding last 22 points ", accuracy )

    print("DEC")
    step = 6 # ____      Jan
    cluster = 22 # possible days of trading in a year data ends at 31 May
    gains, accuracy = evaluate_model_return_gains_accuracy(symbol, model, X_test_scaled, y_test, custom_sample= [step, cluster], X_test=X_test ) # custom_sample= None, last_cluster = None  
    net_gains = gains['net_gains']

    accuracy_by_net_gains = gains['accuracy_by_net_gains']
    accuracy_by_net_gains_before = gains['accuracy_by_net_gains_before']

    print('net profit ', net_gains)
    print('\naccuracy by net gains on previous test data ', accuracy_by_net_gains_before)
    print("accuracy on all test points excluding last 22 points ", accuracy )
    
    
# symbol = "AUDCAD"

# test_model(symbol="USDCAD", iteration=101, learning_rate=0.59, depth=7 ) # Good For Multiple Months
# test_model(symbol="EURUSD", iteration=19, learning_rate=0.49, depth=7 ) # Good For Multiple Months Surplus For Jan, Feb, Mar, Apr
# test_model(symbol="AUDJPY", iteration=11, learning_rate=0.01, depth=5 ) # Good For Multiple Months Surplus For Jan, Feb, Mar, Apr
# test_model(symbol="CHFJPY", iteration=21, learning_rate=0.69, depth=8 ) # Good For Multiple Months Surplus For Jan, Feb, Mar, Apr
# test_model(symbol="EURUSD", iteration=39, learning_rate=0.01, depth=7 )


# Reformed OneZ With Depth 10-11 And Moderate Learning Rate 
# Hour 4 Features Of Last 3 days Only

# ====================
# V2 models
# works when period = 45 in harmonic mean low and harmonic mean high

# test_model(symbol="USDJPY", iteration=3, learning_rate=0.2, depth=12, window_size=7, save_model=True) # version A passed
# test_model(symbol="USDJPY", iteration=3, learning_rate=0.23, depth=12, window_size=7) # version A


# test_model(symbol="EURAUD", iteration=7, learning_rate=0.4, depth=7, window_size=8) # version passed
# test_model(symbol="EURAUD", iteration=33, learning_rate=0.15, depth=10, window_size=8, save_model=True) # version -> A passed

# test_model(symbol="EURAUD", iteration=3, learning_rate=0.15, depth=11, window_size=11) # version 

# xxtest_model(symbol="EURAUD", iteration=27, learning_rate=0.19, depth=10, window_size=8) # another version -> A
# test_model(symbol="EURAUD", iteration=27, learning_rate=0.15, depth=10, window_size=8) # another version -> A
# test_model(symbol="EURAUD", iteration=77, learning_rate=0.5, depth=9, window_size=10) # version 
# test_model(symbol="EURAUD", iteration=100, learning_rate=0.05, depth=6, window_size=9) # version 



# xxtest_model(symbol="EURNZD", iteration=9, learning_rate=0.25, depth=10, window_size=9) # version -> A
# test_model(symbol="EURNZD", iteration=11, learning_rate=0.15, depth=9, window_size=9, save_model=True) # version -> A passed

# test_model(symbol="EURNZD", iteration=66, learning_rate=0.51, depth=10, window_size=9) # another version


# Pair EURNZD__________________________
# test_model(symbol="EURNZD", iteration=5, learning_rate=0.15, depth=10, window_size=6)   # another version
# test_model(symbol="EURNZD", iteration=150, learning_rate=0.02, depth=6, window_size=7) # another version
# test_model(symbol="EURNZD", iteration=300, learning_rate=0.01, depth=6, window_size=7) # another version


#  Pair EURCAD________________________-
# test_model(symbol="EURCAD", iteration=1, learning_rate=0.10, depth=10, window_size=9) # version
# xxtest_model(symbol="EURCAD", iteration=50, learning_rate=0.05, depth=6, window_size=6, alpha_type="gamma" ) # version A gamma 
# test_model(symbol="EURCAD", iteration=13, learning_rate=0.05, depth=9, window_size=7, alpha_type="alpha" ) # version A alpha
# xx test_model(symbol="EURCAD", iteration=5, learning_rate=0.10, depth=10, window_size=9, alpha_type="gamma") # another version A

# test_model(symbol="EURCAD", iteration=22, learning_rate=0.25, depth=8, window_size=9) # version seems passed
# test_model(symbol="EURCAD", iteration=21, learning_rate=0.25, depth=8, window_size=9) # version A passed

# test_model(symbol="EURCAD", iteration=27, learning_rate=0.17, depth=7, window_size=6, alpha_type="beta" ) # version seems passed
# test_model(symbol="EURCAD", iteration=28, learning_rate=0.17, depth=7, window_size=6, alpha_type="beta", save_model=True ) # version A passed






# test_model(symbol="EURUSD", iteration=210, learning_rate=0.05, depth=6, window_size=7, alpha_type="gamma", save_model=True ) #  version A passed
# test_model(symbol="EURUSD", iteration=5, learning_rate=0.05, depth=6, window_size=7, alpha_type="gamma" ) #  another version 
# test_model(symbol="EURUSD", iteration=1, learning_rate=0.25, depth=10, window_size=9, alpha_type="alpha"  ) # another version




# test_model(symbol="USDCAD", iteration=3, learning_rate=0.11, depth=9, window_size=9, alpha_type="omega") # another version 
# test_model(symbol="USDCAD", iteration=1, learning_rate=0.20, depth=10, window_size=9, alpha_type="omega") # another version
# xxtest_model(symbol="USDCAD", iteration=5, learning_rate=0.5, depth=10, window_size=11, alpha_type="gamma") # another version 
# xxtest_model(symbol="USDCAD", iteration=5, learning_rate=0.15, depth=10, window_size=11, alpha_type="gamma") # version A 60 accuracy

# test_model(symbol="USDCAD", iteration=7, learning_rate=0.11, depth=11, window_size=6, alpha_type="delta") # version passed
# test_model(symbol="USDCAD", iteration=8, learning_rate=0.11, depth=11, window_size=6, alpha_type="delta", save_model=True) # version A passed




# xxtest_model(symbol="NZDUSD", iteration=111, learning_rate=0.21, depth=6, window_size=6, alpha_type="gamma") # version A
# test_model(symbol="NZDUSD", iteration=77, learning_rate=0.21, depth=6, window_size=6, alpha_type="gamma") # version A
# test_model(symbol="NZDUSD", iteration=9, learning_rate=0.25, depth=10, window_size=7, alpha_type="gamma") # version A
# test_model(symbol="NZDUSD", iteration=11, learning_rate=0.25, depth=10, window_size=7, alpha_type="gamma", save_model=True) # version A passed

# test_model(symbol="NZDUSD", iteration=1, learning_rate=0.3, depth=7, window_size=7, alpha_type="alpha") # version 
# test_model(symbol="NZDUSD", iteration=3, learning_rate=0.7, depth=11, window_size=7, alpha_type="gamma") # version 
# test_model(symbol="NZDUSD", iteration=11, learning_rate=0.2, depth=10, window_size=7, alpha_type="gamma") # version  


# test_model(symbol="NZDUSD", iteration=17, learning_rate=0.29, depth=10, window_size=7, alpha_type="gamma") # another version
# test_model(symbol="NZDUSD", iteration=21, learning_rate=0.21, depth=6, window_size=7, alpha_type="gamma") # another version
# test_model(symbol="NZDUSD", iteration=3, learning_rate=0.15, depth=9, window_size=7, alpha_type="gamma") # another version




# test_model(symbol="GBPUSD", iteration=3,  learning_rate=0.25, depth=11, window_size=6,  alpha_type="delta") # version 
# test_model(symbol="GBPUSD", iteration=17,  learning_rate=0.25, depth=11, window_size=9,  alpha_type="delta") # another version 
# test_model(symbol="GBPUSD", iteration=5,  learning_rate=0.15, depth=10, window_size=11, alpha_type="gamma") # version A

# test_model(symbol="GBPUSD", iteration=7, learning_rate=0.19, depth=9, window_size=11, alpha_type="gamma") # another version A 
# test_model(symbol="GBPUSD", iteration=19, learning_rate=0.19, depth=9, window_size=11, alpha_type="gamma", save_model=True) # another version A passed
# test_model(symbol="GBPUSD", iteration=9, learning_rate=0.21, depth=9, window_size=11, alpha_type="omega") # another version A




# xxtest_model(symbol="GBPJPY", iteration=5, learning_rate=0.21, depth=9, window_size=7, alpha_type="alpha") # version A 
# test_model(symbol="GBPJPY", iteration=17, learning_rate=0.35, depth=11, window_size=6, alpha_type="alpha") # version A alpha
# test_model(symbol="GBPJPY", iteration=8, learning_rate=0.17, depth=11, window_size=13, alpha_type="delta") # version seems passed

# test_model(symbol="GBPJPY", iteration=8, learning_rate=0.17, depth=11, window_size=13, alpha_type="delta") # version 

# test_model(symbol="GBPJPY", iteration=3, learning_rate=0.35, depth=11, window_size=6, alpha_type="alpha", save_model=True) # version passed close to A

# test_model(symbol="GBPJPY", iteration=7, learning_rate=0.35, depth=11, window_size=6, alpha_type="alpha") # version A passed


# test_model(symbol="GBPJPY", iteration=7, learning_rate=0.15, depth=9, window_size=13, alpha_type="gamma") # another version accuracy by net gains 54-57 
# test_model(symbol="GBPJPY", iteration=2, learning_rate=0.7, depth=13, window_size=6, alpha_type="alpha") # another version 
# test_model(symbol="GBPJPY", iteration=5, learning_rate=0.3, depth=10, window_size=7, alpha_type="alpha") # another version 
# test_model(symbol="GBPJPY", iteration=5, learning_rate=0.3, depth=10, window_size=7, alpha_type="alpha") # another version 
# test_model(symbol="GBPJPY", iteration=7, learning_rate=0.6, depth=12, window_size=11, alpha_type="gamma") # another version 
# test_model(symbol="GBPJPY", iteration=3, learning_rate=0.7, depth=12, window_size=6, alpha_type="alpha") # another version close to A
# test_model(symbol="GBPJPY", iteration=3, learning_rate=0.19, depth=10, window_size=3, alpha_type="delta") # another version 



# test_model(symbol="GBPAUD", iteration=9, learning_rate=0.15, depth=6, window_size=7)
# test_model(symbol="GBPAUD", iteration=5, learning_rate=0.15, depth=9, window_size=11) # another version
# test_model(symbol="GBPAUD", iteration=15, learning_rate=0.3, depth=9, window_size=7)  # another version
# test_model(symbol="GBPAUD", iteration=100, learning_rate=0.05, depth=6, window_size=6)# another version 
# xxtest_model(symbol="GBPAUD", iteration=44, learning_rate=0.05, depth=6, window_size=9) # version A

# test_model(symbol="GBPAUD", iteration=100, learning_rate=0.05, depth=6, window_size=9) # version seems passed
# test_model(symbol="GBPAUD", iteration=12, learning_rate=0.07, depth=5, window_size=6,alpha_type="gamma", save_model=True) # version A passed



# test_model(symbol="AUDJPY", iteration=3, learning_rate=0.2, depth=9, window_size=6, alpha_type="omega") # version omega

# test_model(symbol="AUDJPY", iteration=1, learning_rate=0.45, depth=11, window_size=6, alpha_type="beta") # version 
# test_model(symbol="AUDJPY", iteration=3, learning_rate=0.15, depth=11, window_size=7, alpha_type="beta") # version 

# test_model(symbol="AUDJPY", iteration=1, learning_rate=0.25, depth=11, window_size=7, alpha_type="delta") # version A delta passed
# test_model(symbol="AUDJPY", iteration=1, learning_rate=0.2, depth=11, window_size=7, alpha_type="delta", save_model=True) # version A delta passed

# test_model(symbol="AUDJPY", iteration=27, learning_rate=0.2, depth=10, window_size=11, alpha_type="omega") # version omega

# test_model(symbol="AUDJPY", iteration=1, learning_rate=0.2, depth=11, window_size=9, alpha_type="alpha") # version A
# test_model(symbol="AUDJPY", iteration=1, learning_rate=0.15, depth=13, window_size=7, alpha_type="alpha") # another version
# test_model(symbol="AUDJPY", iteration=3, learning_rate=0.15, depth=12, window_size=7, alpha_type="alpha") # another version


# test_model(symbol="AUDJPY", iteration=3, learning_rate=0.7, depth=12, window_size=6, alpha_type="gamma") # another version
# test_model(symbol="AUDJPY", iteration=1, learning_rate=0.01, depth=9, window_size=9, alpha_type="alpha") # another version
# test_model(symbol="AUDJPY", iteration=5, learning_rate=0.15, depth=13, window_size=7, alpha_type="alpha") # another version


# test_model(symbol="EURJPY", iteration=10, learning_rate=0.7, depth=6, window_size=7, alpha_type="alpha") # another version

# xxtest_model(symbol="EURJPY", iteration=13, learning_rate=0.2, depth=11, window_size=5, alpha_type="gamma") # version A
# test_model(symbol="EURJPY", iteration=11, learning_rate=0.2, depth=11, window_size=5, alpha_type="gamma") # version A 
# test_model(symbol="EURJPY", iteration=5, learning_rate=0.2, depth=11, window_size=5, alpha_type="gamma", save_model=True) # version A passed

# test_model(symbol="EURJPY", iteration=5, learning_rate=0.2, depth=9, window_size=7, alpha_type="delta") # another version

# test_model(symbol="EURJPY", iteration=7, learning_rate=0.5, depth=5, window_size=11, alpha_type="gamma") # another version


# test_model(symbol="AUDNZD", iteration=1, learning_rate=0.5, depth=11, window_size=12, alpha_type="omega") # another version
# test_model(symbol="AUDNZD", iteration=3, learning_rate=0.65, depth=10, window_size=12, alpha_type="omega") # version A

# test_model(symbol="AUDNZD", iteration=3, learning_rate=0.65, depth=10, window_size=12, alpha_type="beta")   # version A
# test_model(symbol="AUDNZD", iteration=3, learning_rate=0.65, depth=10, window_size=12, alpha_type="beta")   # version A

# test_model(symbol="AUDNZD", iteration=13, learning_rate=0.5, depth=9, window_size=9, alpha_type="gamma") # version seems passed
# test_model(symbol="AUDNZD", iteration=5, learning_rate=0.15, depth=11, window_size=7, alpha_type="beta") # version A seems passed
# test_model(symbol="AUDNZD", iteration=9, learning_rate=0.03, depth=10, window_size=6, alpha_type="beta", save_model=True) # version A passed



# test_model(symbol="GBPNZD", iteration=3, learning_rate=0.2, depth=10, window_size=7, alpha_type="omega")
# test_model(symbol="GBPNZD", iteration=3, learning_rate=0.2, depth=9, window_size=6, alpha_type="gamma")
# xxtest_model(symbol="GBPNZD", iteration=13, learning_rate=0.25, depth=10, window_size=7, alpha_type="beta") # version A
# test_model(symbol="GBPNZD", iteration=15, learning_rate=0.25, depth=10, window_size=7, alpha_type="beta") # version A seems passed
# test_model(symbol="GBPNZD", iteration=12, learning_rate=0.25, depth=10, window_size=7, alpha_type="beta", save_model=True) # version A  passed
# test_model(symbol="GBPNZD", iteration=33, learning_rate=0.15, depth=12, window_size=5, alpha_type="beta") # version A


# test_model(symbol="AUDUSD", iteration=5, learning_rate=0.27, depth=10, window_size=9, alpha_type="gamma") # version 
# test_model(symbol="AUDUSD", iteration=5, learning_rate=0.15, depth=12, window_size=5, alpha_type="beta") # version A
# test_model(symbol="AUDUSD", iteration=33, learning_rate=0.15, depth=12, window_size=5, alpha_type="beta") # version A seems passed
# test_model(symbol="AUDUSD", iteration=11, learning_rate=0.15, depth=10, window_size=6, alpha_type="delta") # version 
# test_model(symbol="AUDUSD", iteration=3, learning_rate=0.1, depth=10, window_size=11, alpha_type="delta") # version seems passed
# test_model(symbol="AUDUSD", iteration=7, learning_rate=0.27, depth=10, window_size=9, alpha_type="gamma") # version  
# test_model(symbol="AUDUSD", iteration=23, learning_rate=0.25, depth=8, window_size=7, alpha_type="gamma", save_model=True) # version A passed 

# xxtest_model(symbol="AUDUSD", iteration=21, learning_rate=0.21, depth=9, window_size=9, alpha_type="alpha") # version 
# test_model(symbol="AUDUSD", iteration=14, learning_rate=0.21, depth=7, window_size=9, alpha_type="beta", save_model=True) # version A seems passed
# xx test_model(symbol="AUDUSD", iteration=5, learning_rate=0.15, depth=11, window_size=7, alpha_type="omega") # version 


# test_model(symbol="NZDJPY", iteration=3, learning_rate=0.1, depth=12, window_size=7, alpha_type="beta") # version  

# test_model(symbol="NZDJPY", iteration=5, learning_rate=0.35, depth=11, window_size=7, alpha_type="beta") # version  A
# test_model(symbol="NZDJPY", iteration=11, learning_rate=0.35, depth=11, window_size=7, alpha_type="beta", save_model=True) # version  A passed




# test_model(symbol="USDCHF", iteration=3, learning_rate=0.25, depth=12, window_size=6, alpha_type="gamma") # version passed close to A
# test_model(symbol="USDCHF", iteration=17, learning_rate=0.15, depth=7, window_size=11, alpha_type="beta") # version passed
# test_model(symbol="USDCHF", iteration=7, learning_rate=0.22, depth=8, save_model=False, window_size=6, alpha_type="ha" ) # version passed
# test_model(symbol="USDCHF", iteration=6, learning_rate=0.22, depth=8, window_size=6, alpha_type="ha", save_model=True) # version passed close to A
# xx test_model(symbol="USDCHF", iteration=19, learning_rate=0.19, depth=5, window_size=5, alpha_type="rsi") # version 


# ___ In progress
# test_model(symbol="GBPCAD", iteration=6, learning_rate=0.22, depth=8, window_size=6, alpha_type="ha", save_model=False) # version 


# test_model(symbol="GBPCAD", iteration=13, learning_rate=0.25, depth=8, window_size=7, alpha_type="alpha") # version passed A
# test_model(symbol="GBPCAD", iteration=15, learning_rate=0.25, depth=8, window_size=7, alpha_type="alpha") # version passed close to A

# test_model(symbol="GBPCAD", iteration=35, learning_rate=0.25, depth=8, window_size=5, alpha_type="gamma") # version 


# test_model(symbol="AUDCAD", iteration=23, learning_rate=0.25, depth=8, window_size=5, alpha_type="gamma" ) # # version passed A

# test_model(symbol="CHFJPY", iteration=11, learning_rate=0.75, depth=8, window_size=5, alpha_type="alpha" ) # version close to A

# test_model(symbol="CHFJPY", iteration=21, learning_rate=0.75, depth=8, window_size=5, alpha_type="alpha" ) # version A


# test_model(symbol="NZDCHF", iteration=11, learning_rate=0.77, depth=7, window_size=5, alpha_type="beta") # version close to A
# 
# test_model(symbol="NZDCHF", iteration=22, learning_rate=0.77, depth=7, window_size=5, alpha_type="beta") # version A passed 

# test_model(symbol="NZDCHF", iteration=4, learning_rate=0.5, depth=9, window_size=7, alpha_type="gamma") # version 

# test_model(symbol="NZDCHF", iteration=11, learning_rate=0.15, depth=8, window_size=5, alpha_type="gamma") # version 
# test_model(symbol="EURGBP", iteration=14, learning_rate=0.18, depth=9, window_size=6, alpha_type="omega") # version A passed
# test_model(symbol="EURGBP", iteration=15, learning_rate=0.18, depth=9, window_size=6, alpha_type="omega") # version A passed
# test_model(symbol="EURGBP", iteration=7, learning_rate=0.77, depth=8, window_size=7, alpha_type="gamma") # version close to A

# ____ END
# ____________________________________________

# test_model(symbol="AUDCAD", iteration=23, learning_rate=0.25, depth=8, window_size=5, alpha_type="gamma", save_model=True) # # version passed A

# test_model(symbol="CHFJPY", iteration=21, learning_rate=0.75, depth=8, window_size=5, alpha_type="alpha" , save_model=True) # version A

# test_model(symbol="GBPCAD", iteration=13, learning_rate=0.25, depth=8, window_size=7, alpha_type="alpha", save_model=True) # version passed A


# test_model(symbol="NZDCHF", iteration=22, learning_rate=0.77, depth=7, window_size=5, alpha_type="beta", save_model=True) # version A passed 


# test_model(symbol="EURGBP", iteration=14, learning_rate=0.18, depth=9, window_size=6, alpha_type="omega", save_model=True) # version A passed


# test_model(symbol="CADCHF", iteration=15, learning_rate=0.18, depth=8, window_size=6, alpha_type="beta") # version close to A 

# test_model(symbol="CADCHF", iteration=14, learning_rate=0.20, depth=7, window_size=7, alpha_type="beta") # version close to A

# test_model(symbol="CADCHF", iteration=14, learning_rate=0.20, depth=7, window_size=7, alpha_type="omega", save_model=True) # version A passed


# test_model(symbol="AUDCHF", iteration=25, learning_rate=0.25, depth=7, window_size=11, alpha_type="delta", save_model=True) # version A passed 

# test_model(symbol="AUDCHF", iteration=22, learning_rate=0.25, depth=7, window_size=11, alpha_type="delta") # version close to A

# test_model(symbol="AUDCHF", iteration=27, learning_rate=0.66, depth=8, window_size=6, alpha_type="alpha") # version close to A 


# test_model(symbol="GBPCHF", iteration=13, learning_rate=0.49, depth=7, window_size=5, alpha_type="beta") # version close to A

# test_model(symbol="GBPCHF", iteration=13, learning_rate=0.42, depth=7, window_size=5, alpha_type="beta", save_model=True) # version A passed

# test_model(symbol="GBPCHF", iteration=12, learning_rate=0.42, depth=7, window_size=5, alpha_type="beta") # version 


# test_model(symbol="EURCHF", iteration=19, learning_rate=0.18, depth=7, window_size=5, alpha_type="omega") # version close to A
# test_model(symbol="EURCHF", iteration=20, learning_rate=0.18, depth=7, window_size=5, alpha_type="omega", save_model=True) # version A passed
# test_model(symbol="EURCHF", iteration=12, learning_rate=0.44, depth=7, window_size=5, alpha_type="beta") # version close to A

# test_model( symbol="CADJPY", iteration=7, learning_rate=0.38, depth=9, window_size=11, alpha_type="delta", save_model=True) # version A passed

# test_model( symbol="CADJPY", iteration=7, learning_rate=0.71, depth=12, window_size=6, alpha_type="beta" ) version close to A

# test_model( symbol="CADJPY", iteration=47, learning_rate=0.47, depth=9, window_size=5, alpha_type="beta" ) # version A passed

# test_model( symbol="CADJPY", iteration=7, learning_rate=0.38, depth=9, window_size=11, alpha_type="delta" ) # version A passed

# test_model( symbol="CADJPY", iteration=110, learning_rate=0.27, depth=6, window_size=11, alpha_type="delta" )


# test_model( symbol="NZDCAD", iteration=7, learning_rate=0.2, depth=12, window_size=9, alpha_type="omega" )
# test_model( symbol="NZDCAD", iteration=7, learning_rate=0.2, depth=12, window_size=9, alpha_type="gamma" ) # version close to A 

# test_model( symbol="NZDCAD", iteration=6, learning_rate=0.35, depth=11, window_size=6, alpha_type="gamma" ) # version close to A
# test_model( symbol="NZDCAD", iteration=7, learning_rate=0.36, depth=11, window_size=6, alpha_type="gamma" ) # version close to A

# test_model( symbol="NZDCAD", iteration=7, learning_rate=0.68, depth=11, window_size=6, alpha_type="gamma", save_model=True ) # version A passed


# test_model( symbol="GBPJPY", iteration=27, learning_rate=0.27, depth=7, window_size=8, alpha_type="beta") # version A passed 
# test_model( symbol="GBPJPY", iteration=10, learning_rate=0.1, depth=10, window_size=5, alpha_type="lambda", save_model=True) # version A passed



# __________
# experimental 
# test_model(symbol="USDCHF", iteration=4, learning_rate=0.18, depth=8, window_size=5, alpha_type="beta") # version passed
# test_model(symbol="USDCHF", iteration=3, learning_rate=0.78, depth=13, window_size=7, alpha_type="omega") # version passed close to A
# test_model(symbol="USDCHF", iteration=3, learning_rate=0.78, depth=11, window_size=5, alpha_type="omega") # version passed close to A
# test_model(symbol="USDCHF", iteration=7, learning_rate=0.15, depth=8, window_size=7, alpha_type="ha") # version passed close to A
# test_model(symbol="USDCHF", iteration=3, learning_rate=0.58, depth=11, window_size=5, alpha_type="omega") # version passed 


/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/algorithms.py:1601: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  return arr.searchsorted(value, side=side, sorter=sorter)
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

GBPJPY
0:	learn: 97.5604005	total: 127ms	remaining: 1.14s
9:	learn: 86.7140820	total: 1.23s	remaining: 0us
Mean Squared Error: 9102.590768458482
saved successfully
MAY
GBPJPY  Starting date : 2024-05-01 
['+1-1', '+1-1', '-1+1', 1, 1, 1, 1, 1, 1, '+1-1', '-1+1', '-1+1', '-1+1', '+1-1', 1, -1, 1, 1, 1, '+1-1', '+1-1', '-1+1']
Sharpe Ratio: -0.08
Pips On Profit Side was :  607.1000000000055
Pips On Loss Side was :  726.1999999999972
accuracy by days 50.0
accuracy_by_net_gains  45.533638340958845
net profit  -119.09999999999172

accuracy by net gains on previous test data  56.778204609785696
accuracy on all test points excluding last 22 points  55.61497326203209
APRIL
GBPJPY  Starting date : 2024-04-01 
['-1+1', '-1+1', -1, 1, 1, '-1+1', '+1-1', 1, -1, '-1+1', 1, '-1+1', 1, '+1-1', 1, 1, 1, 1, 1, -1, 1, '+1-1']
Sharpe Ratio: 0.14
Pips On Profit Side was :  1219.800000000009
Pips On Loss Side was :  820.7999999999998
accuracy by days 63.63636363636363
accuracy_by_net_gains  59.776536312849

In [23]:
step = 2
cluster = 22
end, start = 0, 0

for i in range(step): # 1-> 22 + 1 # 2    
        end   += cluster

end += 1
start = end - cluster

print(start)
print(end)


23
45


/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/algorithms.py:1601: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  return arr.searchsorted(value, side=side, sorter=sorter)
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

USDJPY
0:	learn: 70.4186142	total: 554ms	remaining: 1.11s
2:	learn: 65.9572596	total: 1.58s	remaining: 0us
Mean Squared Error: 6107.159143442258
MAY
Sharpe Ratio: 0.37
['-1+1', -1, '-1+1', '-1+1', -1, '+1-1', 1, 1, '+1-1', 1, 1, '-1+1', -1, 1, 1, 1, '+1-1', 1, 1, 1, '+1-1', '+1-1', -1]
Pips On Profit Side was :  1073.9999999999982
Pips On Loss Side was :  353.3000000000044
accuracy by days 60.86956521739131
accuracy_by_net_gains  75.24696980312451
net profit  720.6999999999938

accuracy by net gains on previous test data  62.900529525481666
accuracy on all test points excluding last 22 points  56.68449197860963
APRIL
Sharpe Ratio: 0.64
[-1, 1, 1, 1, '+1-1', 1, 1, 1, '+1-1', '+1-1', -1, 1]
Pips On Profit Side was :  969.199999999995
Pips On Loss Side was :  166.19999999999777
accuracy by days 75.0
accuracy_by_net_gains  85.36198696494638
net profit  802.9999999999973

accuracy by net gains on previous test data  61.7569193742479
accuracy on all test points excluding last 22 points  56.4


# Results Comparison 

In [ ]:




import pandas as pd
import numpy as np

results_dict = {}
day_features, hour4_features = load_features_files()
forex_pairs = [
    'AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'AUDUSD',
    'CADCHF', 'CADJPY',
    'CHFJPY', 
    'EURAUD', 'EURCAD', 'EURCHF', 'EURGBP', 
    'EURJPY', 'EURNZD', 'EURUSD',
    'GBPAUD', 'GBPCAD', 'GBPCHF', 
    'GBPJPY', 'GBPUSD', 'GBPNZD',
    'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDUSD',  
    'USDCHF', 'USDCAD', 'USDJPY'
        ]

symbol_hyperparameter = [
 'AUDJPY', 'AUDUSD', 'AUDCAD', 'AUDCHF', 'AUDNZD',
 'CADJPY', 
 'EURAUD', 'EURCAD', 'EURUSD', 'EURGBP', 'EURNZD',
 'GBPCAD', 'GBPCHF', 'GBPUSD', 'GBPNZD',
 'NZDCHF', 'NZDJPY',
 'USDCHF', 'USDJPY', 'USDCAD',
 'CADCHF', 
 'NZDCAD', 'NZDUSD']

# symbols = ['USDCAD']

for symbol in forex_pairs:
    
    
    day_data, hour4_data = get_day_hour4_features(symbol, day_features, hour4_features)

    X_train_scaled, X_test_scaled, y_train, y_test = get_features_transformed(symbol, day_data, hour4_data)
    
    print(symbol)
    # model = load_model(symbol, path = f"forex_models_custom_hyperparam/{symbol}_model_pips_change")
    # model = load_model(symbol, path =f'forex_models_low_lr_few_h4features_300 iteration/{symbol}_model_pips_change')
    # model = load_model(symbol, path = f"forex_models_regularized")
    model = load_model(symbol, path = f"forex_models_regularized_h4_lag_3")
    
    
    #   

    # if symbol in symbol_hyperparameter: 
      
#        symbol_parameters = load_parameters(symbol)
#        iteration, learning_rate, depth = symbol_parameters['Iterations'], symbol_parameters['Lr'], symbol_parameters['depth']
#        parameters = [ iteration, learning_rate, depth ] 
#        model = train_model(X_train_scaled, X_test_scaled, y_train, y_test, parameters=parameters)
    
#     else:
#       model = train_model(X_train_scaled, X_test_scaled, y_train, y_test)

    # model = load_model(symbol)
    gains , accuracy = evaluate_model_return_gains_accuracy(symbol, model, X_test_scaled,  y_test)
    
     # 2  # 1 --> May 2024 , # 2 --> April 2024
    # step = 1 # ____          May 
    # step = 5 
    
    # step = 4 # February

    # step = 5 # January

    step = 6 # December
    # step = 7 # November
    # step = 8 # October
    
    month_key = {'8': 'oct', '7': 'nov', '6':'dec', '5': 'jan', '4':'feb', '3':'mar', '2':'april', '1':'may' }
    month_name = 'may' if step == 1 else 'april'
    
    try: month_name = month_key[str(step)]
    except : pass
    
    cluster = 22 # possible days of trading in a year data ends at 31 May
    gains, accuracy = evaluate_model_return_gains_accuracy(symbol, model, X_test_scaled, y_test, custom_sample= [step, cluster] ) # custom_sample= None, last_cluster = None  
    
    
    if step == 3 : month_name = 'march' 
    if step == 4 : month_name = 'feb'
    if step == 5 : month_name = 'jan'
    net_gains = gains['net_gains']
    accuracy_by_net_gains = gains['accuracy_by_net_gains']
    # results_dict[symbol] = { 'net_gains': int(net_gains), 'accuracy': int(accuracy), 'month': month_name, 
    #                          'accuracy_by_net_gains': int(accuracy_by_net_gains)  }

    net_gains = gains['net_gains']
    
    accuracy_by_net_gains = gains['accuracy_by_net_gains']
    results_dict[symbol] = { 'net_gains': int(net_gains), 'accuracy': int(accuracy), 'month': month_name, 
                             'accuracy_by_net_gains': int(accuracy_by_net_gains)  }
    # save_model(symbol, model)
    
    # accuracy_by_net_gains = gains['accuracy_by_net_gains_before']
    accuracy_by_net_gains_before = gains['accuracy_by_net_gains_before']

    print('net profit ', net_gains)
    # print('accuracy_by_net_gains \n',accuracy_by_net_gains )
    print('\naccuracy by net gains on previous test data ', accuracy_by_net_gains_before)
    print("accuracy on all test points excluding last 22 points ", accuracy )


## Version 2
* Temporal features True (Month, Day of week, Week Of Month)
* Hour 4 Features Lag by 7
* Day Features (All included) Lag by 9
* Starting Date June 2020 Training Till September 2023
* Ending Date May 2024
* Hour 4 Features 'relative range', 'candle type', 'heikin ashi' 
* Hyperparameters Regularized


Version 1 Regularized On Two Months April May On Two Conditions :
 
 1- hyper parameters chosen should give positive gains for any currency pairs for those two months
 
 2- Previous months accuracy should be 50% and net gains accuracy close to 50% or 52%

Results :- (pips)
* ✅October 2167
* ✅ November 2498
* ❌ December -3080
* ✅ January 4844
* ✅ February 1288
* ✅ March 1883

📉 Not all models were in positive gains in previous month despite all performing good on april may with positive gains

⭐ But parameters performing good on two different months gave them ability to keep majority of currency pairs model in positive gain

💢 December is also not recommended for trading by trading community so we might ignore that month outcomes 





In [ ]:
# For December On Apri-May Month Regularized Data
# print(results_dict)


# Calculate the final sum of net gains
final_sum_net_gain = sum(info['net_gains'] for info in results_dict.values())

# print("Final sum of net gains December: ", final_sum_net_gain)


# Create a list of tuples (symbol, net_gain) and sort it by net_gain in descending order
sorted_list = sorted(results_dict.items(), key=lambda x: x[1]['net_gains'], reverse=False) 
# reverse = False ascending order
# reverse = True descending order


# Convert the sorted list back to a dictionary if needed
sorted_dict = {symbol: info for symbol, info in sorted_list}

# final_sum_net_gain = (  dict(symbol, symbol['pips_loss']) for symbol in pl_info.values())

loss_list = [ (symbol, value['net_gains'])  for symbol, value in sorted_dict.items() ]


print("Most Losses In December: ", loss_list )







## Hyperparameters Only

In [5]:

parameters = {
    'default':{
        'Iterations': 300,
        'Lr': 0.01,
        'scaler': 'RobustScaler()',
        'depth': 6
    },
    
    'NZDJPY': {
        'Iterations': 15,
        'Lr': 0.001,
        'scaler': 'RobustScaler()',
        'depth': 6
    },
    'AUDJPY': {
        'Iterations': 5,
        'Lr': 0.01,
        'scaler': 'RobustScaler()',
        'depth': 6
    },
    'USDJPY': {
        'Iterations': 5,
        'Lr': 0.01,
        'depth': 6
    },
    'CADJPY': {
        'Iterations': 5,
        'Lr': 0.01,
        'depth': 6
    },
    'NZDCAD': {
        'Iterations': 7,
        'Lr': 0.06,
        'scaler': 'RobustScaler()',
        'depth': 8,
        'window_size': 9,
        'accuracy': 52,
        'net pips profit': 114
    },
    'CADCHF': {
        'Iterations': 7,
        'Lr': 0.06,
        'scaler': 'RobustScaler()',
        'depth': 8,
        'window_size': 9,
        'accuracy': 52,
        'net pips profit': 158
    },
    'USDCHF2': {
        'Iterations': 7,
        'Lr': 0.06,
        'scaler': 'RobustScaler()',
        'depth': 8,
        'window_size': 9,
        'accuracy': 50,
        'net pips profit': 179
    },
    'EURAUD': {
        'Iterations': 7,
        'Lr': 0.06,
        'scaler': 'RobustScaler()',
        'depth': 8,
        'window_size': 9,
        'accuracy': 56,
        'net pips profit': 306
    },
    'EURNZD3': {
        'Iterations': 15,
        'Lr': 0.1,
        'scaler': 'RobustScaler()',
        'depth': 7,
        'window_size': 9,
        'accuracy': 52,
        'net pips profit': 278
    },
    'GBPNZD': {
        'Iterations': 15,
        'Lr': 0.1,
        'scaler': 'RobustScaler()',
        'depth': 7,
        'window_size': 9,
        'accuracy': 50,
        'net pips profit': 224
    },
    'NZDCHF2': {
        'Iterations': 15,
        'Lr': 0.1,
        'scaler': 'RobustScaler()',
        'depth': 7,
        'window_size': 9,
        'accuracy': 50,
        'net pips profit': 140
    },
    'EURUSD3': {
        'Iterations': 15,
        'Lr': 0.1,
        'scaler': 'RobustScaler()',
        'depth': 7,
        'window_size': 9,
        'accuracy': 56,
        'net pips profit': 306
    },
    'GBPCAD': {
        'Iterations': 5,
        'Lr': 0.001,
        'scaler': 'RobustScaler()',
        'depth': 7,
        'window_size': 9,
        'accuracy': 54,
        'net pips profit': 151
    },
    'GBPUSD2': {
        'Iterations': 5,
        'Lr': 0.001,
        'scaler': 'RobustScaler()',
        'depth': 7,
        'window_size': 9,
        'accuracy': 52,
        'net pips profit': 217
    },
    'EURGBP': {
        'Lr': 0.001 ,
        'Iterations': 188,
        'depth': 7,
        'accuracy' :52,
        'pips net profit' : 97
    },
    'EURCAD2': {
        'Iterations': 15,
        'Lr': 0.1,
        'scaler': 'RobustScaler()',
        'depth': 7,
        'window_size': 9,
        'accuracy': 51,
        'pips net profit':  228
    }, 
    'GBPCHF': {
        'Iterations': 15,
        'Lr': 0.1,
        'scaler': 'RobustScaler()',
        'depth': 8,
        'window_size': 9,
        'accuracy': 53,
        'pips net profit':  241
    }, 
    'NZDUSD': {
        'Iterations': 5, 
        'Lr': 0.77, 
        'depth': 7,
        'accuracy': 54
    }, 
    'EURUSD2': {
        'Lr':0.025,
        'Iterations': 450, 
        'depth': 5,
        'accuracy': 53
    },
    'EURNZD2':{
     'Iterations': 29, 
     'Lr': 0.63, 
     'depth': 7,
     'accuracy': 50
    }, 
    'GBPUSD': {
     'Iterations': 13, 
     'Lr': 0.77, 
     'depth': 7,
     'accuracy': 53
    },
    'GBPAUD': {
    'Iterations': 13,
    'Lr': 0.77,
    'depth': 8,
    'accuracy': 55       
    },
    'AUDUSD2': {
    'Iterations': 5,
    'Lr': 0.7,
    'depth': 7,
    'accuracy': 50       
    },
    'AUDCHF': {
    'Iterations': 7,
    'Lr': 0.71,
    'depth' : 7
    }, 
    'AUDCAD':{
    'Iterations' : 9,
    'Lr': 0.67,
    'depth': 7
    },
    'AUDNZD':{
    'Iterations': 15,
    'Lr': 0.68,
    'depth': 7 
    }, 
    'EURCAD':{ 
    'Iterations': 20,
    'Lr': 0.039,
    'depth': 7
    },
    'NZDCHF' : {
    'Iterations': 27,
    'Lr' : 0.35,
    'depth' : 7
    },
    'USDCAD': {
    'Iterations': 20,
    'Lr': 0.039,
    'depth': 7
    },
    'USDCHF':{
    'Iterations': 188,
    'Lr': 0.67,
    'depth': 7
    },
    'NZDCAD':{
    'Iterations': 17,
    'Lr': 0.38,
    'depth': 7
    }, 
    'AUDUSD' :{
    'Iterations': 11 ,
    'Lr': 0.211,
    'depth': 7
    },
    'EURUSD':{
    'Iterations': 23,
    'Lr': 0.51,
    'depth': 7
    },
    'EURNZD':{
    'Iterations': 188,
    'Lr' :0.55,
    'depth': 7
    }
}





# Python Code To Load Train And Evaluate Forex Model

In [213]:


import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import RobustScaler, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier
from catboost import Pool
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import randint


def get_features_transformed(symbol, day_data, hour4_data, date_resume = '2020-06-01', X_test_re = False, temporal=False, window_size = 9, alpha_type= "gamma"):
    
    
    # date_resume
    # split_date = pd.to_datetime('2020-06-01') 
    
    split_date = pd.to_datetime(date_resume)

    day_data = day_data.loc[split_date:]


    # split_date = pd.to_datetime('2020-06-01')

    hour4_data = hour4_data.loc[split_date:]


    day_data['pips_change'] = np.subtract( day_data['Close'].to_numpy(), day_data['Close'].shift(1).to_numpy() )
    
    if 'JPY' in symbol: pips_change = day_data['pips_change'] * 10 ** 2 
    
    
    else :  pips_change = day_data['pips_change'] * 10 ** 4 
    
    day_data['Target'] = pips_change
    
    
    # features =  [f'candle_type_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'rsi_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'Price_Range_T-{i}' for i in range(1, window_size + 1)  ] +\
    #            [f'rsi_crossover_slow_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'rsi_crossover_fast_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'harmonic_mean_low_T-{i}' for i in range(1, window_size + 1)  ] +\
    #            [f'harmonic_mean_high_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'slow_harmonic_mean_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'fast_harmonic_mean_T-{i}' for i in range(1, window_size + 1) ]
             
    # features = [f'Day_of_Week_T-{i}' for i in range(1, window_size + 1)] + [f'Week_of_Month_T-{i}' for i in range(1, window_size + 1)] +\
    #            [f'Month_T-{i}' for i in range(1, window_size + 1)]  +\
    #            [f'High_T-{i}' for i in range(1, window_size + 1) ] +  [f'Low_T-{i}'  for i in range(1, window_size + 1) ] + \
    #            [f'STDEV_T-{i}' for i in range(1, window_size + 1) ] + \
    #            [f'Upper_Band_T-{i}' for i in range(1, window_size + 1) ] + \
    #            [f'Lower_Band_T-{i}' for i in range(1, window_size + 1) ]+ \
    #            [f'candle_type_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'rsi_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'Price_Range_T-{i}' for i in range(1, window_size + 1)  ] +\
    #            [f'Median_Price_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'rsi_crossover_slow_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'rsi_crossover_fast_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'harmonic_mean_low_T-{i}' for i in range(1, window_size + 1)  ] +\
    #            [f'harmonic_mean_high_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'slow_harmonic_mean_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'fast_harmonic_mean_T-{i}' for i in range(1, window_size + 1) ] 
             
             
    # features = [f'Day_of_Week_T-{i}' for i in range(1, window_size + 1)] + [f'Week_of_Month_T-{i}' for i in range(1, window_size + 1)] +\
    #            [f'Month_T-{i}' for i in range(1, window_size + 1)]  +\
    #            [f'High_T-{i}' for i in range(1, window_size + 1) ] +  [f'Low_T-{i}'  for i in range(1, window_size + 1) ] + \
    #            [f'STDEV_T-{i}' for i in range(1, window_size + 1) ] + \
    #            [f'RSI_T-{i}' for i in range(1, window_size + 1)   ] + \
    #            [f'Upper_Band_T-{i}' for i in range(1, window_size + 1) ] + \
    #            [f'Lower_Band_T-{i}' for i in range(1, window_size + 1) ]+ \
    #            [f'candle_type_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'heikin_ashi_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'Price_Range_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'Median_Price_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'ema_3_T-{i}' for i in range(1, window_size + 1) ]  +\
    #            [f'ema_5_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'ema_7_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'ema_14_T-{i}'for i in range(1, window_size + 1) ] +\
    #            [f'ema_difference_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'supertrend_status_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'supertrend_crossover_T-{i}' for i in range(1, window_size + 1) ] + \
    #            [f'supertrend_T-{i}' for i in range(1, window_size + 1) ] +\
    #            [f'elastic_supertrend_T-{i}' for i in range(1, window_size + 1) ] + \
    #            [f'elastic_supertrend_cross_T-{i}' for i in range(1, window_size + 1) ] + \
    #            [f'elastic_supertrend_status_T-{i}' for i in range(1, window_size + 1) ]

    if temporal == False : 
        # [f'Day_of_Week_T-{i}' for i in range(1, window_size + 1)]  +
        features =  [f'candle_type_T-{i}' for i in range(1, window_size + 1) ] +\
               [f'rsi_T-{i}' for i in range(1, window_size + 1) ] +\
               [f'Price_Range_T-{i}' for i in range(1, window_size + 1)  ] +\
               [f'rsi_crossover_slow_T-{i}' for i in range(1, window_size + 1) ] +\
               [f'rsi_crossover_fast_T-{i}' for i in range(1, window_size + 1) ] +\
               [f'harmonic_mean_low_T-{i}' for i in range(1, window_size + 1)  ] +\
               [f'harmonic_mean_high_T-{i}' for i in range(1, window_size + 1) ] +\
               [f'slow_harmonic_mean_T-{i}' for i in range(1, window_size + 1) ] +\
               [f'fast_harmonic_mean_T-{i}' for i in range(1, window_size + 1) ] 
        
    day_features_X   =  day_data[features]

    day_features_X = day_features_X.apply(pd.to_numeric, errors='coerce')

    day_features_X = day_features_X.astype(float)

    hour4_features_X = hour4_data

    hour4_features_X = hour4_features_X.apply(pd.to_numeric, errors='coerce')

    hour4_features_X = hour4_features_X.astype(float)
    
    # substrings = ['relative_range', 'candle_type', 'heikin_ashi', 'stdev', 'true' ] #, 'heikin_ashi',  'supertrend_h4']

    # substrings = ['relative_range', 'candle_type', 'supertrend_h4' ] #, 'heikin_ashi',  'supertrend_h4']
    # substrings = [ 'supertrend_h4', 'candle_type', 'crossover', 'slow_harmonic', 'fast_harmonic' ]
    # substrings = [ 'supertrend_h4' ]
    
    if alpha_type == "gamma":
        substrings = [ 'supertrend_h4' ]

    elif alpha_type == "beta":
        substrings = [ 'supertrend_h4', 'candle_type']
        
        
    elif alpha_type == "omega":
        substrings = [ 'supertrend_h4', 'candle_type', 'rsi_slow_crossover']
        
    elif alpha_type == "alpha":
        substrings = [ 'relative_range', 'fast_harmonic_mean']
    
    elif alpha_type == "delta":
        substrings = [ 'relative_range', 'rsi_slow_crossover', 'candle_type', 'supertrend_h4']
    
    elif alpha_type == "ha":
        substrings = [ 'stdev_slow', 'heikin_ashi', 'slow_harmonic_mean'  ] # 'slow_harmonic_mean'
    
    elif alpha_type == "rsi":
        substrings = [ 'rsi_sma_slow', 'candle_type', 'slow_harmonic_mean', 'supertrend_h4' ] # 'slow_harmonic_mean' ]
        # features += [f'High_T-{i}' for i in range(1, window_size + 1) ] +
        # [f'High_T-{i}' for i in range(1, window_size + 1) ] +  [f'Low_T-{i}'  for i in range(1, window_size + 1) ]
    elif alpha_type == "lambda":
        substrings = ['relative_range', 'candle_type', 'heikin_ashi' ]

    # substrings = [ 'supertrend_h4', 'candle_type' ]
    # substrings = [ 'supertrend_h4' ] 
    # # Filter columns that contain any of the specified substrings
    filtered_columns = [col for col in hour4_features_X.columns if any(sub in col for sub in substrings)]
    hour4_features_X = hour4_features_X[filtered_columns]
    # hour4_features_X = hour4_features_X.filter(like='relative_range')

    X = day_features_X.join(hour4_features_X)
    # X = day_features_X
    
    # if temporal == False :  X = day_features_X
    
    # X = day_features_X.join(hour4_features_X)

    y = day_data['Target']

    X.fillna(0.0, inplace=True)

    y.fillna(0.0, inplace=True)

    # print(len(data[features]))
    X = X.astype(float)

    div = int(len(X) * 0.8)
    # train = :div
    # test  = div:
    X_train = X[:div]
    X_test  = X[div:]

    y_train = y[:div]
    y_test  = y[div:]

    # # Apply RobustScaler
    scaler = RobustScaler()
    
    # scaler = StandardScaler()

    X_train_scaled = scaler.fit_transform(X_train)
    # X_test_scaled = scaler.transform(X_test)
    X_test_scaled = scaler.fit_transform(X_test)
    
    if not X_test_re:
        return X_train_scaled, X_test_scaled, y_train, y_test
    else :
        return X_train_scaled, X_test_scaled, y_train, y_test, X_test

def train_model_2(X_train_scaled, X_test_scaled, y_train, y_test ):
    
    # # Convert to Pool (CatBoost-specific data structure)
    train_pool = Pool(X_train_scaled, y_train)
    test_pool = Pool(X_test_scaled, y_test)
    
    
    np.random.seed(42)


    # Initialize CatBoost regressor
    model = CatBoostRegressor(iterations=300, early_stopping_rounds=75, 
                              learning_rate=0.1 , depth=5, verbose=100)

    # Train the model using train pool
    model.fit(train_pool)


    # Make predictions using test pool
    y_pred = model.predict(X_test_scaled)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    
    print(f'Mean Squared Error: {mse}')
    
    return model

def finetune_model(X_train_scaled, X_test_scaled, y_train, y_test, parameters = None ):

#     model = CatBoostRegressor()
#     parameters = {'depth' : [6,8,10],'learning_rate' : [0.01, 0.05, 0.1],
#                   'iterations'    : [30, 50, 100]}

#     model = GridSearchCV(estimator=model, param_grid = parameters, cv = 2, n_jobs=-1)
#     model.fit(X_train_scaled, y_train)
    
#     # Make predictions using test pool
#     y_pred = model.predict(X_test_scaled)

#     # Evaluate the model
#     mse = mean_squared_error(y_test, y_pred)
    
#     print(f'Mean Squared Error: {mse}')

        # Create the model
    rf_regressor = RandomForestRegressor(random_state=42)

    # Define the parameter distribution
    param_dist = {
        'n_estimators': randint(50, 200),
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': randint(2, 11),
        'min_samples_leaf': randint(1, 5),
        'bootstrap': [True, False]
    }

    # Create the RandomizedSearchCV object
    model = RandomizedSearchCV(estimator=rf_regressor, param_distributions=param_dist,
                                       n_iter=100, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error',
                                       random_state=42)

    # Fit the random search to the data
    model.fit(X_train_scaled, y_train)

    # Get the best parameters and best score
    best_params = random_search.best_params_
    best_score  = random_search.best_score_

    # Make predictions using test pool
    y_pred = model.predict(X_test_scaled)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    
    print(f'Mean Squared Error: {mse}')
    
    print("Best Parameters: ",best_params)
    print("Best Score: ",best_score)
    
    return model
    
    
def train_model(X_train_scaled, X_test_scaled, y_train, y_test, parameters = None ):
    
    # train_model_1
    # # Convert to Pool (CatBoost-specific data structure)
    train_pool = Pool(X_train_scaled, y_train)
    test_pool = Pool(X_test_scaled, y_test)
    
    
    # np.random.seed(42)


    # Initialize CatBoost regressor
    if not parameters:
        model = CatBoostRegressor(iterations=300, early_stopping_rounds=75, 
                              learning_rate=0.01 , depth=6, verbose=100)
    
    else : 
        iteration, learning_rate, depth = parameters[0], parameters[1], parameters[2]
        model = CatBoostRegressor(iterations=iteration, early_stopping_rounds=75, 
                              learning_rate=learning_rate, depth=depth, verbose=100)
        
    # Train the model using train pool
    model.fit(train_pool)


    # Make predictions using test pool
    y_pred = model.predict(X_test_scaled)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    
    print(f'Mean Squared Error: {mse}')
    
    return model

    
    
def train_model_1(X_train_scaled, X_test_scaled, y_train, y_test ):
    
    # train_model_1
    # # Convert to Pool (CatBoost-specific data structure)
    train_pool = Pool(X_train_scaled, y_train)
    test_pool = Pool(X_test_scaled, y_test)
    
    
    np.random.seed(42)


    # Initialize CatBoost regressor
    model = CatBoostRegressor(iterations=300, early_stopping_rounds=75, 
                              learning_rate=0.01 , depth=6, verbose=100)

    # Train the model using train pool
    model.fit(train_pool)


    # Make predictions using test pool
    y_pred = model.predict(X_test_scaled)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    
    print(f'Mean Squared Error: {mse}')
    
    return model


# def load_features_files(hour4_lag_3=None):

#     # day_features_path = 'day_features_data_lagby_14_v2.bin'

#     with open(day_features_path, 'rb') as file :

#         day_features = pickle.load(file)


#     # hour4_features_path = 'hour4_features_data.bin'  
#     # hour4_features_path = 'hour4_features_data_lag_by_7.bin'
#     hour4_features_path = 'hour4_features_data_lag_by_3_v2.bin'
    
#     if hour4_lag_3 == True : hour4_features_path = 'hour4_features_data_lag_by_3_v2.bin'
     
#     with open(hour4_features_path, 'rb') as file :

#         hour4_features = pickle.load(file)
    
#     return  day_features, hour4_features
    

def evaluate_model_old(symbol, model, X_test_scaled,  y_test):
    
    sample = 0
    right  = 0
    wrong  = 0
    status = []
    pips_profit = 0 
    pips_loss = 0

    for point in range(1, 22 + 1 ):

        sample += 1
        y_pred = model.predict(X_test_scaled[-point])
        y_true = y_test[-point]

        if y_pred < 0.0 and y_true < 0.0 :
            pips_profit += abs(y_true)
            right += 1
            status.append(1)

        elif y_pred > 0.0 and y_true > 0.0 :
            pips_profit += abs(y_true)
            right += 1
            status.append(1)

        else:
            pips_loss += abs(y_true)
            wrong += 1
            status.append(0)

    # y_pred = model.predict(test_pool )

        # print("actual : ",y_test[-3],"\npredicted : ",y_pred )
    print(f"{symbol}\n")
    print("accuracy on last 22 test points Month May 2024 with low learning rate 2020 dataset 80% training dataset is ",right/sample * 100 )

    # print("accuracy on last 100 test points is ",right) #wrong/sample * 100 )
    print(status)
    print(f"Pips On Profit Side was : ",pips_profit)
    print("Pips On Loss Side was : ",pips_loss)

    print("net pips profit : ", pips_profit-pips_loss )
    
    
    status = []
    for point in range(1, len(X_test_scaled) + 1 ):

        sample += 1
        y_pred = model.predict(X_test_scaled[-point])
        y_true = y_test[-point]

        if y_pred < 0.0 and y_true < 0.0 :
            # pips_profit += abs(y_true)
            right += 1
            status.append(1)

        elif y_pred > 0.0 and y_true > 0.0 :
            # pips_profit += abs(y_true)
            right += 1
            status.append(1)

        else:
            # pips_loss += abs(y_true)
            wrong += 1
            status.append(0)

    print("accuracy on all test points with low learning rate 2020 dataset 80% training dataset is ",right/sample * 100 )

    # print("accuracy on last 100 test points is ",right) #wrong/sample * 100 )
    print(status)
    
    
    print(f"{symbol}\n==============================")
    

def save_model_v2(symbol, model, parameters, alpha_type, window_size):
    
    import pickle
    
    # parameters = [ iteration, learning_rate, depth ]
    iterations = parameters[0]
    learning_rate = parameters[1] 
    depth = parameters[2]
    model_file = {"model": model ,  "model_info": { "learning_rate": learning_rate, "depth": depth, 
                  "iterations": iterations, "alpha_type": alpha_type, "window_size": window_size,
                  "scaling": "robust_scaling", "version": "v2" }, "symbol": symbol+"_pips_change" }
    with open(f"v2_forex_models/{symbol}_model_pips_change", 'wb') as file:
        
        pickle.dump(model_file, file)
    print("saved successfully")
        
    # model.save_model(f"v2_forex_models/{symbol}_model_pips_change")

    
    
def save_model(symbol, model):
    
    model.save_model(f"v2_forex_models/{symbol}_model_pips_change")

    
    
def load_features_files():

    day_features_path = 'day_features_data_lagby_14_v2.bin' 
    # day_features_path = 'day_features_data_lagby_14_v3.bin'


    with open(day_features_path, 'rb') as file :

        day_features = pickle.load(file)


    # hour4_features_path = 'hour4_features_data.bin'  
    hour4_features_path = 'hour4_features_data_lag_by_3_v2.bin'

    with open(hour4_features_path, 'rb') as file :

        hour4_features = pickle.load(file)
        
    return  day_features, hour4_features
    
    
def get_day_hour4_features(symbol, day_features, hour4_features):
    
    
    
    for day_pair in day_features: 
        if  day_pair['symbol'] == symbol: 

            day_data = day_pair['day_features'] 
            break


    for hour4_pair in hour4_features: 
        if  hour4_pair['symbol'] == symbol: 

            hour4_data = hour4_pair['hour4_features'] 
            break
    
    return  day_data, hour4_data

def load_model(symbol, path = None):
    
    
    if path: 
        pass
        path = f"{path}/{symbol}_model_pips_change"
        
    else: 
        path = f"v2_forex_models/{symbol}_model_pips_change"
    
    model = CatBoostRegressor()
    
    model.load_model(path)
    return model


# def load_model_old(symbol):
    
#     pass
#     model = CatBoostRegressor()
#     path = f'forex_models_low_lr_few_h4features_300 iteration/{symbol}_model_pips_change'
#     model.load_model(path)
#     return model


def evaluate_model(symbol, model, X_test_scaled, y_test):
    
    sample = 0
    right  = 0
    wrong  = 0
    status = []
    pips_profit = 0 
    pips_loss = 0

    for point in range(23, 45 + 1 ): # may 1 - 22 # april 23 - 55

        sample += 1
        y_pred = model.predict(X_test_scaled[-point])
        y_true = y_test[-point]

        if y_pred < 0.0 and y_true < 0.0 :
            pips_profit += abs(y_true)
            right += 1
            status.append(1)

        elif y_pred > 0.0 and y_true > 0.0 :
            pips_profit += abs(y_true)
            right += 1
            status.append(1)

        else:
            pips_loss += abs(y_true)
            wrong += 1
            status.append(0)

    # y_pred = model.predict(test_pool )

        # print("actual : ",y_test[-3],"\npredicted : ",y_pred )
    print(f"{symbol}\n")
    print("accuracy on last 22 test points Month May 2024 with low learning rate 2020 dataset 80% training dataset is ",right/sample * 100 )

    # print("accuracy on last 100 test points is ",right) #wrong/sample * 100 )
    print(status)
    print(f"Pips On Profit Side was : ",pips_profit)
    print("Pips On Loss Side was : ",pips_loss)

    print("net pips profit : ", pips_profit-pips_loss )
    
    status = []
    for point in range(1, len(X_test_scaled) + 1 ):

        sample += 1
        y_pred = model.predict(X_test_scaled[-point])
        y_true = y_test[-point]

        if y_pred < 0.0 and y_true < 0.0 :
            # pips_profit += abs(y_true)
            right += 1
            status.append(1)

        elif y_pred > 0.0 and y_true > 0.0 :
            # pips_profit += abs(y_true)
            right += 1
            status.append(1)

        else:
            # pips_loss += abs(y_true)
            wrong += 1
            status.append(0)

    print("accuracy on all test points excluding last 22 points with low learning rate 2020 dataset 80% training dataset is ",right/sample * 100 )

    # print("accuracy on last 100 test points is ",right) #wrong/sample * 100 )
    print(status)
    
    
    print(f"{symbol}\n==============================")
    # try:
    #     return  pips_profit-pips_loss, right/sample * 100
    # except : pass

    # net_gains , accuracy
    
def evaluate_model_return_gains_accuracy(symbol, model, X_test_scaled, y_test, custom_sample= None, X_test = None ):
    
    sample = 0
    right  = 0
    wrong  = 0
    status = []
    pips_profit = 0
    pips_loss = 0
    
    # custom_sample=[1,22]
    if custom_sample:
        step, cluster = custom_sample[0] , custom_sample[1] 
        start , end = 0 , 0
        # for i in range(step):
        #     start += 1 + end # 23
        #     end   += cluster      # 
        
        for i in range(step): # 1-> 22 + 1 # 2
            end   += cluster
        end += 1
        start = end - cluster
        
    else : start, end = 1, 22
    
#     # Convert net gains to numpy array
# returns = np.array(net_gains)

# # Calculate mean return
# mean_return = np.mean(returns)

# # Calculate standard deviation of returns
# std_return = np.std(returns)

# # Assuming risk-free rate is 0 for simplicity
# risk_free_rate = 0

# # Calculate Sharpe ratio
# sharpe_ratio = (mean_return - risk_free_rate) / std_return

# print(f'Sharpe Ratio: {sharpe_ratio:.2f}')


    net_gains_list = []
    for point in range(start, end  ): # may 1 - 22 # april 23 - 45

        sample += 1
        y_pred = model.predict(X_test_scaled[-point])
        y_true = y_test[-point]

        if y_pred < 0.0 and y_true < 0.0 :
            pips_profit += abs(y_true)
            right += 1
            net_gains_list.append(abs(y_true))
            status.append(-1)

        elif y_pred > 0.0 and y_true > 0.0 :
            pips_profit += abs(y_true)
            right += 1
            net_gains_list.append(y_true)
            status.append(1)

        else:
            pips_loss += abs(y_true)
            wrong += 1
            # status.append(0)
            net_gains_list.append(-1*abs(y_true))

            if y_pred > 0.0 and y_true < 0.0 : status.append(str('+1-1'))

            elif y_pred < 0.0 and y_true > 0.0 : status.append(str('-1+1'))
        

    # y_pred = model.predict(test_pool )
    
    # print("actual : ",y_test[-3],"\npredicted : ",y_pred )
    # print(f"{symbol}\n")
    # print("accuracy on last 22 test points Month May 2024 with low learning rate 2020 dataset 80% training dataset is ",right/sample * 100 )
    
    returns = np.array(net_gains_list)

    # Calculate mean return
    mean_return = np.mean(returns)

    # Calculate standard deviation of returns
    std_return = np.std(returns)

    # Assuming risk-free rate is 0 for simplicity
    risk_free_rate = 0

    # Calculate Sharpe ratio
    sharpe_ratio = (mean_return - risk_free_rate) / std_return
    if len(X_test) != None :
        date   = X_test.index.values[-end]
        date = str(date.astype('datetime64[D]'))
        print(symbol , f" Starting date : {date} ")
        
    status = status[::-1] # starting from end because end is the first most day of the chosen month
    
    print(status)
    # print("accuracy on last 100 test points is ",right) #wrong/sample * 100 )
    accuracy_by_days = right/sample * 100
    # print(f"Accuracy by days :",accuracy_by_days)
    print(f'Sharpe Ratio: {sharpe_ratio:.2f}')
    
    print(f"Pips On Profit Side was : ",pips_profit)
    print("Pips On Loss Side was : ",pips_loss)
    accuracy_by_net_gains = pips_profit / (pips_profit+pips_loss)*100
    print('accuracy by days', accuracy_by_days )
    
    print('accuracy_by_net_gains ',accuracy_by_net_gains )
    net_gains = pips_profit-pips_loss
#     print("net pips profit : ", pips_profit-pips_loss )
    
    status = []
    right, wrong = 0, 0
    sample = 0
    pips_profit, pips_loss = 0, 0
    for point in range(end, len(X_test_scaled) + 1 ): # excluding May April 55  # excluding May 23 -> till end of length

        sample += 1
        y_pred = model.predict(X_test_scaled[-point])
        y_true = y_test[-point]

        if y_pred < 0.0 and y_true < 0.0 :
            pips_profit += abs(y_true)
            right += 1
            status.append(-1)

        elif y_pred > 0.0 and y_true > 0.0 :
            pips_profit += abs(y_true)
            right += 1
            status.append(1)

        else:
            pips_loss += abs(y_true)
            wrong += 1
            # status.append(0)
            if y_pred > 0.0 and y_true < 0.0 : status.append(str('+1-1'))

            elif y_pred < 0.0 and y_true > 0.0 : status.append(str('-1+1'))

    # print("accuracy on all test points excluding last 22 points with low learning rate 2020 dataset 80% training dataset is ",right/sample * 100 )

    # print("accuracy on last 100 test points is ",right) #wrong/sample * 100 )
    # print(status)
    accuracy_by_days_before = right/sample * 100
    accuracy_by_net_gains_before = pips_profit / (pips_profit+pips_loss)*100
    gains = {}
    gains['net_gains'] = net_gains
    gains['accuracy_by_days']  = accuracy_by_days
    gains['accuracy_by_days_before']  = accuracy_by_days_before
    gains['accuracy_by_net_gains_before'] = accuracy_by_net_gains_before
    gains['accuracy_by_net_gains'] = accuracy_by_net_gains
    gains['sharpe_ratio'] = sharpe_ratio
    
    # print(f"{symbol}\n==============================")
    try:
        return  gains, accuracy_by_days_before
    except : pass




def update_profit_and_loss(pl_info, date, pips_profit, pips_loss, net_gain):
    if date in pl_info:
        pl_info[date]['pips_profit'] += pips_profit
        pl_info[date]['pips_loss'] += pips_loss
        pl_info[date]['net_gain'] += net_gain
    else:
        pl_info[date] = {'pips_profit': pips_profit, 'pips_loss': pips_loss, 'net_gain': net_gain}

    return pl_info
        

def each_day_gain_loss(day_features, hour4_features, custom_sample= None , model_path = None):

    pass
    # custom_sample = [ 1, 22 ]  
    forex_pairs = [
    'AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'AUDUSD',
    'CADCHF', 'CADJPY',
    'CHFJPY', 
    'EURAUD', 'EURCAD', 'EURCHF', 'EURGBP', 
    'EURJPY', 'EURNZD', 'EURUSD',
    'GBPAUD', 'GBPCAD', 'GBPCHF', 
    'GBPJPY', 'GBPUSD', 'GBPNZD',
    'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDUSD',  
    'USDCHF', 'USDCAD', 'USDJPY']
    symbols_store = {}
    
    sample = 0
    right  = 0
    wrong  = 0
    status = []
    pips_profit = 0
    pips_loss = 0
    pl_info = {}
    # custom_sample=[1,22]
    if custom_sample :
        step, cluster = custom_sample[0] , custom_sample[1] 
        start , end = 0 , 0
        for i in range(step):
            end   += cluster + 1

        start = end - cluster
    
    
    else : start, end = 1, 22
    
    
    for point in range(start, end + 1 ): # may 1 - 22 # april 23 - 45
        
        pips_profit, pips_loss = 0, 0
        net_gains = 0
        
        sample += 1
        for symbol in forex_pairs:
    
            
            if symbol not in symbols_store:
            
                day_data, hour4_data = get_day_hour4_features(symbol, day_features, hour4_features)
                X_train_scaled, X_test_scaled, y_train, y_test, X_test = get_features_transformed(symbol, day_data, hour4_data, date_resume='2020-06-01',  X_test_re=True)
                
                if model_path: model = load_model(symbol, path = model_path)
                else : model = load_model(symbol)
                
                
                symbols_store[symbol] = { 'X_test_scaled': X_test_scaled,  
                                         'y_test': y_test, 'model': model, 'X_test': X_test  }
                # symbols_store[symbol] = { 'y_test': y_test }
                # symbols_store[symbol] = { 'model': model }
                # symbols_store[symbol] = { 'X_test': X_test }
            
            else:
                X_test_scaled = symbols_store[symbol]['X_test_scaled']
                y_test = symbols_store[symbol]['y_test']
                model  = symbols_store[symbol]['model']
                X_test = symbols_store[symbol]['X_test']
            
            y_pred = model.predict(X_test_scaled[-point])
            y_true = y_test[-point]
            date   = X_test.index.values[-point]
            # date = np.datetime64(date)
            date = str(date.astype('datetime64[D]'))
            
            if y_pred < 0.0 and y_true < 0.0 :
                pips_profit += abs(y_true)
                right += 1
                status.append(-1)

            elif y_pred > 0.0 and y_true > 0.0 :
                pips_profit += abs(y_true)
                right += 1
                status.append(1)

            else:
                pips_loss += abs(y_true)
                wrong += 1
                # status.append(0)
                if y_pred > 0.0 and y_true < 0.0 : status.append(str('+1-1'))

                elif y_pred < 0.0 and y_true > 0.0 : status.append(str('-1+1'))
            
        net_gain = int(pips_profit - pips_loss)
        pl_info  =  update_profit_and_loss(pl_info, date, int(pips_profit), int(pips_loss), net_gain)


    return pl_info


def update_symbol_profit_and_loss(pl_info, date, pips_profit, pips_loss, net_gain, symbol):
      
#     symbol
    # if 'date_info' not in pl_info:  pl_info['date_info'] = date
#     if symbol in pl_info:
#         pl_info[symbol]['pips_profit'] += pips_profit
#         pl_info[symbol]['pips_loss'] += pips_loss
#         pl_info[symbol]['net_gain'] += net_gain
        
#     else:
#         # pl_info[date] = {'pips_profit': pips_profit, 'pips_loss': pips_loss, 'net_gain': net_gain}
    pl_info[symbol] = {'pips_profit': pips_profit, 'pips_loss': pips_loss, 'net_gain': net_gain, 'date': date}

        
    return pl_info

# '2024-01-29': {'pips_profit': 137, 'pips_loss': 1127, 'net_gain': -989} 



def each_symbol_gain_loss_by_date(day_features, hour4_features, custom_sample= None , model_path = None, date_val='2024-01-29'):

    pass
    # custom_sample = [ 1, 22 ]  
    forex_pairs = [
    'AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'AUDUSD',
    'CADCHF', 'CADJPY',
    'CHFJPY', 
    'EURAUD', 'EURCAD', 'EURCHF', 'EURGBP', 
    'EURJPY', 'EURNZD', 'EURUSD',
    'GBPAUD', 'GBPCAD', 'GBPCHF', 
    'GBPJPY', 'GBPUSD', 'GBPNZD',
    'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDUSD',  
    'USDCHF', 'USDCAD', 'USDJPY']
    symbols_store = {}
    
    sample = 0
    right  = 0
    wrong  = 0
    status = []
    pips_profit = 0
    pips_loss = 0
    pl_info = {}
    # custom_sample=[1,22]
    if custom_sample :
        step, cluster = custom_sample[0] , custom_sample[1] 
        start , end = 0 , 0
        for i in range(step):
            end   += cluster + 1

        start = end - cluster
    
    
    else : start, end = 1, 22
    
    
    for point in range(start, end + 1 ): # may 1 - 22 # april 23 - 45
        
        pips_profit, pips_loss = 0, 0
        net_gains = 0
        
        sample += 1
        for symbol in forex_pairs:
            
            pips_profit, pips_loss = 0, 0
            if symbol not in symbols_store:
            
                day_data, hour4_data = get_day_hour4_features(symbol, day_features, hour4_features)
                X_train_scaled, X_test_scaled, y_train, y_test, X_test = get_features_transformed(symbol, day_data, hour4_data, date_resume='2020-06-01',  X_test_re=True)
                
                if model_path: model = load_model(symbol, path = model_path)
                else : model = load_model(symbol)
                
                
                symbols_store[symbol] = { 'X_test_scaled': X_test_scaled,  
                                         'y_test': y_test, 'model': model, 'X_test': X_test  }
                # symbols_store[symbol] = { 'y_test': y_test }
                # symbols_store[symbol] = { 'model': model }
                # symbols_store[symbol] = { 'X_test': X_test }
            
            else:
                X_test_scaled = symbols_store[symbol]['X_test_scaled']
                y_test = symbols_store[symbol]['y_test']
                model  = symbols_store[symbol]['model']
                X_test = symbols_store[symbol]['X_test']
            
            
            date   = X_test.index.values[-point]
            # date = np.datetime64(date)
            date = str(date.astype('datetime64[D]'))
            
            if date != date_val : break
            
            y_pred = model.predict(X_test_scaled[-point])
            y_true = y_test[-point]
            
            if y_pred < 0.0 and y_true < 0.0 :
                pips_profit = abs(y_true)
                right += 1
                status.append(-1)

            elif y_pred > 0.0 and y_true > 0.0 :
                pips_profit = abs(y_true)
                right += 1
                status.append(1)

            else:
                pips_loss = abs(y_true)
                wrong += 1
                # status.append(0)
                if y_pred > 0.0 and y_true < 0.0 : status.append(str('+1-1'))

                elif y_pred < 0.0 and y_true > 0.0 : status.append(str('-1+1'))
            
            net_gain = int(pips_profit - pips_loss) # net gain on symbol basis for a particular day
            pl_info  =  update_symbol_profit_and_loss(pl_info, date, int(pips_profit), int(pips_loss), net_gain, symbol)


    return pl_info



def each_day_gain_loss_with_threshold(day_features, hour4_features, custom_sample= None , model_path = None, threshold = 5 ):

    pass
    # custom_sample = [ 1, 22 ]  
    forex_pairs = [
    'AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'AUDUSD',
    'CADCHF', 'CADJPY',
    'CHFJPY', 
    'EURAUD', 'EURCAD', 'EURCHF', 'EURGBP', 
    'EURJPY', 'EURNZD', 'EURUSD',
    'GBPAUD', 'GBPCAD', 'GBPCHF', 
    'GBPJPY', 'GBPUSD', 'GBPNZD',
    'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDUSD',  
    'USDCHF', 'USDCAD', 'USDJPY']
    symbols_store = {}
    
    sample = 0
    right  = 0
    wrong  = 0
    status = []
    pips_profit = 0
    pips_loss = 0
    pl_info = {}
    # custom_sample=[1,22]
    if custom_sample :
        step, cluster = custom_sample[0] , custom_sample[1] 
        start , end = 0 , 0
        for i in range(step):
            end   += cluster + 1

        start = end - cluster
    
    
    else : start, end = 1, 22
    
    era = start + 10
    
    for point in range(start, end + 1 ): # may 1 - 22 # april 23 - 45
        
        pips_profit, pips_loss = 0, 0
        net_gains = 0
        
        sample += 1
        for symbol in forex_pairs:
    
            
            if symbol not in symbols_store:
            
                day_data, hour4_data = get_day_hour4_features(symbol, day_features, hour4_features)
                X_train_scaled, X_test_scaled, y_train, y_test, X_test = get_features_transformed(symbol, day_data, hour4_data, date_resume='2020-06-01',  X_test_re=True)
                
                if model_path: model = load_model(symbol, path = model_path)
                else : model = load_model(symbol)
                
                
                symbols_store[symbol] = { 'X_test_scaled': X_test_scaled,  
                                         'y_test': y_test, 'model': model, 'X_test': X_test  }
                # symbols_store[symbol] = { 'y_test': y_test }
                # symbols_store[symbol] = { 'model': model }
                # symbols_store[symbol] = { 'X_test': X_test }
            
            else:
                X_test_scaled = symbols_store[symbol]['X_test_scaled']
                y_test = symbols_store[symbol]['y_test']
                model  = symbols_store[symbol]['model']
                X_test = symbols_store[symbol]['X_test']
            
            y_pred = model.predict(X_test_scaled[-point])
            y_true = y_test[-point]
            date   = X_test.index.values[-point]
            # date = np.datetime64(date)
            date = str(date.astype('datetime64[D]'))
            if point < era : 
                if abs(y_pred) < threshold : continue
            
            if y_pred < 0.0 and y_true < 0.0 :
                pips_profit += abs(y_true)
                right += 1
                status.append(-1)

            elif y_pred > 0.0 and y_true > 0.0 :
                pips_profit += abs(y_true)
                right += 1
                status.append(1)

            else:
                pips_loss += abs(y_true)
                wrong += 1
                # status.append(0)
                if y_pred > 0.0 and y_true < 0.0 : status.append(str('+1-1'))

                elif y_pred < 0.0 and y_true > 0.0 : status.append(str('-1+1'))
            
        net_gain = int(pips_profit - pips_loss)
        pl_info  =  update_profit_and_loss(pl_info, date, int(pips_profit), int(pips_loss), net_gain)


    return pl_info

#     accuracy_by_days = right/sample * 100

#     accuracy_by_net_gains = pips_profit / (pips_profit+pips_loss)*100

#     net_gains = pips_profit-pips_loss

    
import pickle

parameters = {
    'NZDJPY': {
        'Iterations': 15,
        'Lr': 0.001,
        'scaler': 'RobustScaler()',
        'depth': 6
    },
    'AUDJPY': {
        'Iterations': 5,
        'Lr': 0.01,
        'scaler': 'RobustScaler()',
        'depth': 6
    },
    'USDJPY': {
        'Iterations': 5,
        'Lr': 0.01,
        'depth': 6
    },
    'CADJPY': {
        'Iterations': 5,
        'Lr': 0.01,
        'depth': 6
    },
    'NZDCAD': {
        'Iterations': 7,
        'Lr': 0.06,
        'scaler': 'RobustScaler()',
        'depth': 8,
        'window_size': 9,
        'accuracy': 52,
        'net pips profit': 114
    },
    'CADCHF': {
        'Iterations': 7,
        'Lr': 0.06,
        'scaler': 'RobustScaler()',
        'depth': 8,
        'window_size': 9,
        'accuracy': 52,
        'net pips profit': 158
    },
    'USDCHF2': {
        'Iterations': 7,
        'Lr': 0.06,
        'scaler': 'RobustScaler()',
        'depth': 8,
        'window_size': 9,
        'accuracy': 50,
        'net pips profit': 179
    },
    'EURAUD': {
        'Iterations': 7,
        'Lr': 0.06,
        'scaler': 'RobustScaler()',
        'depth': 8,
        'window_size': 9,
        'accuracy': 56,
        'net pips profit': 306
    },
    'EURNZD3': {
        'Iterations': 15,
        'Lr': 0.1,
        'scaler': 'RobustScaler()',
        'depth': 7,
        'window_size': 9,
        'accuracy': 52,
        'net pips profit': 278
    },
    'GBPNZD': {
        'Iterations': 15,
        'Lr': 0.1,
        'scaler': 'RobustScaler()',
        'depth': 7,
        'window_size': 9,
        'accuracy': 50,
        'net pips profit': 224
    },
    'NZDCHF2': {
        'Iterations': 15,
        'Lr': 0.1,
        'scaler': 'RobustScaler()',
        'depth': 7,
        'window_size': 9,
        'accuracy': 50,
        'net pips profit': 140
    },
    'EURUSD3': {
        'Iterations': 15,
        'Lr': 0.1,
        'scaler': 'RobustScaler()',
        'depth': 7,
        'window_size': 9,
        'accuracy': 56,
        'net pips profit': 306
    },
    'GBPCAD': {
        'Iterations': 5,
        'Lr': 0.001,
        'scaler': 'RobustScaler()',
        'depth': 7,
        'window_size': 9,
        'accuracy': 54,
        'net pips profit': 151
    },
    'GBPUSD2': {
        'Iterations': 5,
        'Lr': 0.001,
        'scaler': 'RobustScaler()',
        'depth': 7,
        'window_size': 9,
        'accuracy': 52,
        'net pips profit': 217
    },
    'EURGBP': {
        'Lr': 0.001 ,
        'Iterations': 188,
        'depth': 7,
        'accuracy' :52,
        'pips net profit' : 97
    },
    'EURCAD2': {
        'Iterations': 15,
        'Lr': 0.1,
        'scaler': 'RobustScaler()',
        'depth': 7,
        'window_size': 9,
        'accuracy': 51,
        'pips net profit':  228
    }, 
    'GBPCHF': {
        'Iterations': 15,
        'Lr': 0.1,
        'scaler': 'RobustScaler()',
        'depth': 8,
        'window_size': 9,
        'accuracy': 53,
        'pips net profit':  241
    }, 
    'NZDUSD': {
        'Iterations': 5, 
        'Lr': 0.77, 
        'depth': 7,
        'accuracy': 54
    }, 
    'EURUSD2': {
        'Lr':0.025,
        'Iterations': 450, 
        'depth': 5,
        'accuracy': 53
    },
    'EURNZD2':{
     'Iterations': 29, 
     'Lr': 0.63, 
     'depth': 7,
     'accuracy': 50
    }, 
    'GBPUSD': {
     'Iterations': 13, 
     'Lr': 0.77, 
     'depth': 7,
     'accuracy': 53
    },
    'GBPAUD': {
    'Iterations': 13,
    'Lr': 0.77,
    'depth': 8,
    'accuracy': 55       
    },
    'AUDUSD2': {
    'Iterations': 5,
    'Lr': 0.7,
    'depth': 7,
    'accuracy': 50       
    },
    'AUDCHF': {
    'Iterations': 7,
    'Lr': 0.71,
    'depth' : 7
    }, 
    'AUDCAD':{
    'Iterations' : 9,
    'Lr': 0.67,
    'depth': 7
    },
    'AUDNZD':{
    'Iterations': 15,
    'Lr': 0.68,
    'depth': 7 
    }, 
    'EURCAD':{ 
    'Iterations': 20,
    'Lr': 0.039,
    'depth': 7
    },
    'NZDCHF' : {
    'Iterations': 27,
    'Lr' : 0.35,
    'depth' : 7
    },
    'USDCAD': {
    'Iterations': 20,
    'Lr': 0.039,
    'depth': 7
    },
    'USDCHF':{
    'Iterations': 188,
    'Lr': 0.67,
    'depth': 7
    },
    'NZDCAD':{
    'Iterations': 17,
    'Lr': 0.38,
    'depth': 7
    }, 
    'AUDUSD' :{
    'Iterations': 11 ,
    'Lr': 0.211,
    'depth': 7
    },
    'EURUSD':{
    'Iterations': 23,
    'Lr': 0.51,
    'depth': 7
    },
    'EURNZD':{
    'Iterations': 188,
    'Lr' :0.55,
    'depth': 7
    }
}

# with open('parameters', 'wb') as file:
#     pickle.dump(parameters, file )
    

def load_parameters(symbol):
    
    with open('parameters', 'rb') as file:
        parameters = pickle.load(file )
        
    return parameters[symbol]
        

## Python Code To Make A Binary File From All Daily OHLC Currency


In [4]:
import pandas as pd
import numpy as np


forex_pairs = [
    'AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'AUDUSD',
    'CADCHF', 'CADJPY',
    'CHFJPY', 
    'EURAUD', 'EURCAD', 'EURCHF', 'EURGBP', 
    'EURJPY', 'EURNZD', 'EURUSD',
    'GBPAUD', 'GBPCAD', 'GBPCHF', 
    'GBPJPY', 'GBPUSD', 'GBPNZD',
    'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDUSD',  
    'USDCHF', 'USDCAD', 'USDJPY'
        ]


currency_ohlc = []
count = 0
for pair in forex_pairs:
    daily_path = f'currency_data/{pair}_Daily.csv'
    data = return_df_day(daily_path)
    data = add_features(data)
    data = make_features_lagged( lag_by= 14)

    currency_df = { "day_features": data, "symbol": f"{pair}"  }
    currency_ohlc.append(currency_df)
    count += 1
    print(count)
    


import pickle

with open('day_features_data_lagby_14_v3.bin', 'wb') as file:
    pickle.dump(currency_ohlc, file)
    






/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:124: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:125: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all column

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


In [15]:
import pandas as pd
import numpy as np


forex_pairs = [
    'AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'AUDUSD',
    'CADCHF', 'CADJPY',
    'CHFJPY', 
    'EURAUD', 'EURCAD', 'EURCHF', 'EURGBP', 
    'EURJPY', 'EURNZD', 'EURUSD',
    'GBPAUD', 'GBPCAD', 'GBPCHF', 
    'GBPJPY', 'GBPUSD', 'GBPNZD',
    'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDUSD',  
    'USDCHF', 'USDCAD', 'USDJPY'
        ]

currency_ohlc = []
count = 0
for pair in forex_pairs:
    daily_path = f'currency_data/{pair}_Daily.csv'
    data = return_df_day(daily_path)
    data = add_features(data)
    break

New Harmonic Mean Date
2013-06-03         NaN
2013-06-04         NaN
2013-06-05         NaN
2013-06-06         NaN
2013-06-07         NaN
                ...   
2024-04-29   -0.002816
2024-04-30   -0.002596
2024-05-01   -0.002531
2024-05-02   -0.002480
2024-05-03   -0.002668
Name: High, Length: 2837, dtype: float64
Old Harmonic Mean Date
2013-06-03    0.016441
2013-06-04    0.016441
2013-06-05    0.016441
2013-06-06    0.016441
2013-06-07    0.016441
                ...   
2024-04-29    0.016441
2024-04-30    0.016441
2024-05-01    0.016441
2024-05-02    0.016441
2024-05-03    0.016441
Name: harmonic_mean_high, Length: 2837, dtype: float64


In [2]:

# Calculate RSI
def calculate_rsi(series, period=10):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))


def ema( price, period):

  price = np.array(price)
  alpha = 2 / (period + 1.0)
  alpha_reverse = 1 - alpha
  data_length = len(price)

  power_factors = alpha_reverse ** (np.arange(data_length + 1))
  initial_offset = price[0] * power_factors[1:]

  scale_factors = 1 / power_factors[:-1]

  weight_factor = alpha * alpha_reverse ** (data_length - 1)

  weighted_price_data = price * weight_factor * scale_factors
  cumulative_sums = weighted_price_data.cumsum()
  ema_values = initial_offset + cumulative_sums * scale_factors[::-1]

  return ema_values
    




def candle_type(o, h, l, c):

    diff = abs(c - o)
    o1, c1 = np.roll(o, 1), np.roll(c, 1)  #
    min_oc = np.where(o < c, o, c)
    max_oc = np.where(o > c, o, c)

    pattern = np.where(
      np.logical_and( min_oc - l > diff, h - max_oc < diff), 6,
      np.where(np.logical_and( h - max_oc > diff, min_oc - l < diff),
      4, np.where(np.logical_and(np.logical_and(c > o, c1 < o1), np.logical_and(c > o1, o < c1)),
        5, np.where( min_oc - l > diff, 3,
                      np.where(np.logical_and( h - max_oc > diff,
                  min_oc - l < diff),
                      2, np.where(np.logical_and(np.logical_and(c > o, c1 < o1), np.logical_and(c > o1, o < c1)),
                      1, 0))))))
    return pattern


def heikin_ashi_status( ha_open, ha_close):

    candles = np.full_like(ha_close, '', dtype='U10')

    for i in range(1, len(ha_close)):

        if ha_close[i] > ha_open[i]: candles[i] = 2 #'Green'

        elif ha_close[i] < ha_open[i]: candles[i] = 1 # 'Red'

        else: candles[i] = 0 #'Neutral'

    return candles

def heikin_ashi_candles( open, high, low, close):

    ha_low, ha_close = np.empty(len(close), dtype=np.float32), np.empty(len(close), dtype=np.float32)
    ha_open, ha_high = np.empty(len(close), dtype=np.float32), np.empty(len(close), dtype=np.float32)

    ha_open[0] = (open[0] + close[0]) / 2
    ha_close[0] = (close[0] + open[0] + high[0] + low[0]) / 4

    for i in range(1, len(close)):
        ha_open[i] = (ha_open[i - 1] + ha_close[i - 1]) / 2
        ha_close[i] = (open[i] + high[i] + low[i] + close[i]) / 4
        ha_high[i] = max(high[i], ha_open[i], ha_close[i])
        ha_low[i] = min(low[i], ha_open[i], ha_close[i])

    return ha_open, ha_close, ha_high, ha_low


def true_range( high, low, close):

  close_shift = shift(close, 1)
  high_low, high_close, low_close = np.array(high - low, dtype=np.float32), \
                                    np.array(abs(high - close_shift), dtype=np.float32), \
                                    np.array(abs(low - close_shift), dtype=np.float32)

  true_range = np.max(np.hstack((high_low, high_close, low_close)).reshape(-1, 3), axis=1)

  return true_range



def shift(array, place):

  array = np.array(array, dtype=np.float32)
  shifted = np.roll(array, place)
  shifted[0:place] = np.nan
  shifted[np.isnan(shifted)] = np.nanmean(shifted)

  return shifted


def ma_based_supertrend_indicator( high, low, close, atr_length=10, atr_multiplier=3, ma_length=10):

    # Calculate True Range and Smoothed ATR
    tr = true_range(high, low, close)
    atr = ema(tr, atr_length)

    upper_band = (high + low) / 2 + (atr_multiplier * atr)
    lower_band = (high + low) / 2 - (atr_multiplier * atr)

    trend = np.zeros(len(atr))

    # Calculate Moving Average
    ema_values = ema(close, ma_length)

    if ema_values[0] > lower_band[0]:
        trend[0] = lower_band[0]
    elif ema_values[0] < upper_band[0]:
        trend[0] = upper_band[0]
    else:
        trend[0] = upper_band[0]

    # Compute final upper and lower bands
    for i in range(1, len(close)):
        if ema_values[i] > trend[i - 1]:
            trend[i] = max(trend[i - 1], lower_band[i])


        elif ema_values[i] < trend[i - 1]:
            trend[i] = min(trend[i - 1], upper_band[i])

        else:
            trend[i] = trend[i - 1]

    status_value = np.where(ema_values > trend, 1.0, -1.0)

    return trend, status_value




def supertrend_status_crossover( status_value):


    prev_status = np.roll(status_value, 1)
    supertrend_status_crossover = np.where((prev_status < 0) & (status_value > 0), 1.0, np.where((prev_status > 0) & (status_value < 0), -1.0, 0))

    return supertrend_status_crossover




def supertrend_indicator(high, low, close, period, multiplier=1.0):

    true_range_value = true_range(high, low, close)

    smoothed_atr = ema(true_range_value, period)

    upper_band = (high + low) / 2 + (multiplier * smoothed_atr)
    lower_band = (high + low) / 2 - (multiplier * smoothed_atr)

    supertrend = np.zeros(len(true_range_value))
    trend = np.zeros(len(true_range_value))

    if close[0] > upper_band[0]: trend[0] = upper_band[0]
    elif close[0] < lower_band[0]: trend[0] = lower_band[0]
    else:  trend[0] = upper_band[0]

    for i in range(1, len(close)):

        if close[i] > upper_band[i]: trend[i] = upper_band[i]
        elif close[i] < lower_band[i]: trend[i] = lower_band[i]
        else: trend[i] = trend[i - 1]

    # Calculate Buy/Sell Signals using numpy where  # np.where( close > trend, '1 Buy', '-1 Sell')
    status_value = np.where(close > trend, 1.0, -1.0)

    return trend, status_value

def supertrend_status_crossover(status_value):

    prev_status = np.roll(status_value, 1)
    supertrend_status_crossover = np.where((prev_status < 0) & (status_value > 0), 1.0, np.where((prev_status > 0) & (status_value < 0), -1.0, 0))

    return supertrend_status_crossover


def rsi_crossover(smoothed_rsi):
    
    prev_signal = np.roll(smoothed_rsi, 1)
    direction, crossover = direction_crossover_signal_line(smoothed_rsi, prev_signal)
    
    return crossover
    pass

def rsi_crossover_with_sma(rsi , slow_smoothed_rsi): # 10 period sma of actual rsi
    
    direction = np.where(rsi - slow_smoothed_rsi > 0, 1, -1)
    
    prev_direction = np.roll(direction, 1)
    crossover = np.where((prev_direction == -1) & (direction == 1), 1,
                             np.where((prev_direction == 1) & (direction == -1), -1, 0))
    
    return crossover

def calculate_harmonic_mean(series, period=3):
    def harmonic_mean(s):
        n = len(s)
        return n / sum(1 / x for x in s if x != 0) if n > 0 else 0
    
    return series.rolling(window=period).apply(harmonic_mean, raw=True)


def calculate_hm_high(series, period=3):

    def harmonic_mean_high(high):
        n = len(high)
        prev_high = np.roll(high, 1)    
        diff = high - prev_high

        hm = n / sum(1 / x for x in diff if x != 0)
        y = np.sin(hm)

        y_pi = y * np.pi
        return y_pi
    return  series.rolling(window=period).apply(harmonic_mean_high, raw=True)

def harmonic_mean_high(high):
        n = len(high)
        prev_high = np.roll(high, 1)    
        diff = high - prev_high

        hm = n / sum(1 / x for x in diff if x != 0)
        y = np.sin(hm)

        y_pi = y * np.pi
        return y_pi


def calculate_hm_low(series, period=3):

    def harmonic_mean_low(low):
        n = len(low)
        prev_low = np.roll(low, 1)    
        diff = prev_low - low

        hm = n / sum(1 / x for x in diff if x != 0)
        y = np.sin(hm)

        y_pi = y * np.pi
        return y_pi
    return  series.rolling(window=period).apply(harmonic_mean_low, raw=True)
    

def direction_crossover_signal_line(signal, prev_signal):

        direction = np.where(signal - prev_signal > 0, 1, -1) # current bigger then upward direction , if current small then downward direction
        prev_direction = np.roll(direction, 1)
        crossover = np.where((prev_direction == -1) & (direction == 1), 1,
                             np.where((prev_direction == 1) & (direction == -1), -1, 0))

        return direction, crossover

In [3]:
def add_features(data):
    
    open_value, high, low, close = data['Open'], data['High'], data['Low'], data['Close']
    # high , low = data['High'].to_numpy(), data['Low'].to_numpy()
    
#     elastic_supertrend, es_status_value = ma_based_supertrend_indicator( high, low, close, atr_length=10, atr_multiplier=2.5, ma_length=10)

#     elastic_supertrend_crossover = supertrend_status_crossover(es_status_value)

    # supertrend, supertrend_status_value = supertrend_indicator(high, low, close, period= 10, multiplier=0.66)
    # supertrend_crossover = supertrend_status_crossover(supertrend_status_value)

    candle_type_value  = candle_type(open_value, high, low, close)

    # ha_open, ha_close, ha_high, ha_low = heikin_ashi_candles(open_value, high, low, close)
    # heikin_ashi_candle = heikin_ashi_status(ha_open, ha_close)
    hl2 = ( data['High'] + data['Low'] ) / 2
    
    rsi  = calculate_rsi(hl2, period=10)
    smoothed_rsi =  rsi.rolling(window=3).mean()
    slow_smoothed_rsi =  rsi.rolling(window=10).mean()
    
    data['rsi_sma_fast'] = smoothed_rsi
    data['rsi_sma_slow'] = slow_smoothed_rsi
    
    data['rsi'] =  rsi
    data['rsi_crossover_fast'] =  rsi_crossover(smoothed_rsi)
    data['rsi_crossover_slow'] =  rsi_crossover_with_sma(rsi , slow_smoothed_rsi)
    
    
    data['STDEV'] = data['Close'].rolling(window=5).std()
    data['Upper_Band'] = data['Close'].rolling(window=5).mean() + (data['Close'].rolling(window=5).std() * 2)
    data['Lower_Band'] = data['Close'].rolling(window=5).mean() - (data['Close'].rolling(window=5).std() * 2)

    data['candle_type'] = candle_type_value
    
    data['slow_harmonic_mean'] = calculate_harmonic_mean(data['Close'], period=9)
    data['fast_harmonic_mean'] = calculate_harmonic_mean(data['Close'], period=3)
    
    data['harmonic_mean_high'] = calculate_hm_high(high)
    # data['harmonic_mean_high'] = harmonic_mean_high(high)
    # print("New Harmonic Mean",calculate_hm_high(high)[:-20])
    # print("Old Harmonic Mean",data['harmonic_mean_high'][:-20])
    
    data['harmonic_mean_low']  = calculate_hm_low(low)

    data['Price_Range'] = data['High'] - data['Low']
    data['Median_Price'] = (data['High'] + data['Low']) / 2
    
    data['daily_returns'] = data['Close'] - data['Close'].shift(1)
    
    data['Day_of_Week'] = data.index.dayofweek + 1  # Monday=1, ..., Friday=5
    data['Week_of_Month'] = (data.index.day - 1) // 7 + 1
    data['Month'] = data.index.month

    return data

#     data['supertrend']  = np.asarray(supertrend)
#     data['supertrend_status']  = np.asarray(supertrend_status_value)


#     data['supertrend_crossover'] = np.asarray(supertrend_crossover)
#     data['supertrend_value'] = np.asarray(supertrend)

#     data['elastic_supertrend'] = elastic_supertrend
#     data['elastic_supertrend_status'] = es_status_value
#     data['elastic_supertrend_cross'] = elastic_supertrend_crossover

#     data['heikin_ashi'] = heikin_ashi_candle

#     ema_3  = ema( (high+low/2), 3 )
#     ema_5  = ema( (high+low/2), 5 )
#     ema_7  = ema( (high+low/2), 7 )
#     ema_14 = ema( (high+low/2), 14)

#     # Add seasonality features
#     data['Day_of_Week'] = data.index.dayofweek + 1  # Monday=1, ..., Friday=5
#     data['Week_of_Month'] = (data.index.day - 1) // 7 + 1
#     data['Month'] = data.index.month

    # Add numerical features
    # data['Prev_Close'] = data['Close'].shift(1)
    
    
    
#     data['ema_3']  = ema_3
#     data['ema_5']  = ema_5
#     data['ema_7']  = ema_7
#     data['ema_14'] = ema_14

#     selected_columns = data[['ema_3', 'ema_5', 'ema_7' ]]

#     ema_mean = np.mean(selected_columns, axis=1)
#     data['ema_difference'] = pd.Series(np.subtract( ema_mean, ema_14 ))




def make_features_lagged( lag_by= 9):
    
    # Create window-based features
    window_size = lag_by

    for i in range(1, window_size + 1):
        data[f'Day_of_Week_T-{i}']   = data['Day_of_Week'].shift(i)
        data[f'Week_of_Month_T-{i}'] = data['Week_of_Month'].shift(i)
        data[f'Month_T-{i}'] = data['Month'].shift(i)
        data[f'Close_T-{i}'] = data['Close'].shift(i)
        data[f'Open_T-{i}'] = data['Open'].shift(i)
        data[f'High_T-{i}'] = data['High'].shift(i)
        data[f'Low_T-{i}']  = data['Low'].shift(i)
        data[f'STDEV_T-{i}']   = data['STDEV'].shift(i)
        data[f'rsi_T-{i}']  = data['rsi'].shift(i)
        data[f'rsi_crossover_fast_T-{i}']  = data['rsi_crossover_fast'].shift(i)
        data[f'rsi_crossover_slow_T-{i}']  = data['rsi_crossover_slow'].shift(i)
        
        data[f'Price_Range_T-{i}']   = data['Price_Range'] .shift(i)
        data[f'Median_Price_T-{i}']  = data['Median_Price'].shift(i)
        data[f'Upper_Band_T-{i}']  = data['Upper_Band'].shift(i)
        data[f'Lower_Band_T-{i}']  = data['Lower_Band'].shift(i)
        data[f'candle_type_T-{i}'] = data['candle_type'].shift(i)
        data[f'slow_harmonic_mean_T-{i}'] = data['slow_harmonic_mean'].shift(i)
        data[f'fast_harmonic_mean_T-{i}'] = data['fast_harmonic_mean'].shift(i)
        data[f'harmonic_mean_high_T-{i}'] = data['harmonic_mean_high'].shift(i)
        data[f'harmonic_mean_low_T-{i}'] = data['harmonic_mean_low'].shift(i)
        data[f'daily_returns_T-{i}'] = data['daily_returns'].shift(i)

    return data
        
        
#         data[f'supertrend_crossover_T-{i}'] = data['supertrend_crossover'].shift(i)
#         data[f'elastic_supertrend_T-{i}']   = data['elastic_supertrend'].shift(i)
#         data[f'elastic_supertrend_status_T-{i}']   = data['elastic_supertrend_status'].shift(i)

#         data[f'elastic_supertrend_cross_T-{i}'] = data['elastic_supertrend_cross'].shift(i)


        # data[f'ema_3_T-{i}']  = data['ema_3'].shift(i)
        # data[f'ema_5_T-{i}']  = data['ema_5'].shift(i)
        # data[f'ema_7_T-{i}']  = data['ema_7'].shift(i)
        # data[f'ema_14_T-{i}'] = data['ema_14'].shift(i)
        # data[f'ema_difference_T-{i}'] = data['ema_difference'].shift(i)
        # data[f'supertrend_value_T-{i}'] = data['supertrend_value'].shift(i)




def return_df_day(daily_path):
    
    data = pd.read_csv(daily_path, sep='\t')

    # Rename columns as requested
    data.rename(columns={
    '<DATE>': 'Date',
    '<OPEN>': 'Open',
    '<HIGH>': 'High',
    '<LOW>': 'Low',
    '<CLOSE>': 'Close',
    '<TICKVOL>': 'TickVol',
    '<VOL>': 'Vol',
    '<SPREAD>': 'Spread'
    }, inplace=True)

    data['Date'] = pd.to_datetime(data['Date'], format='%Y.%m.%d')

    # Set Date column as the index
    data.set_index('Date', inplace=True)

    data = data[['Open', 'High', 'Low', 'Close']]

    return data




## Python Code To Make A Binary File From All Hour4 OHLC Currency


In [8]:
forex_pairs = [ 'AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'AUDUSD',
    'CADCHF', 'CADJPY',
    'CHFJPY', 
    'EURAUD', 'EURCAD', 'EURCHF', 'EURGBP', 
    'EURJPY', 'EURNZD', 'EURUSD',
    'GBPAUD', 'GBPCAD', 'GBPCHF', 
    'GBPJPY', 'GBPUSD', 'GBPNZD',
    'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDUSD',  
    'USDCHF', 'USDCAD', 'USDJPY' ]


currency_ohlc = []
count = 0
for pair in forex_pairs:
    # daily_path = f'currency_data/{pair[:6]}_Daily.csv'
    hour4_path = f'currency_data/{pair}_H4.csv'

    data_h4     = rename_h4_df(hour4_path)

    reshaped_h4 = combine_ohlc_into_single_day(data_h4)

    features_h4 = add_ohlc_in_lagged(reshaped_h4,  lag_by= 3 )

    features_h4 = add_features(features_h4,  lag_by= 3)
    
    currency_df = { "hour4_features": features_h4, "symbol": f"{pair}"  }
    
    currency_ohlc.append(currency_df)
    count += 1
    print(count)

print(count)




import pickle

with open('hour4_features_data_lag_by_3_v2.bin', 'wb') as file:
    pickle.dump(currency_ohlc, file)
    

# import pickle

# with open('hour4_less_features_data_lag_by_3.bin', 'wb') as file:
#     pickle.dump(currency_ohlc, file)
    

    
    
    

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:115: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:124: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all column

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
28


In [7]:

# WITH LESS FEATURES FOR HOUR 4 


def rename_h4_df(hour4_path):

    data_h4 = pd.read_csv(hour4_path, sep='\t')

    # Rename columns as requested
    data_h4.rename(columns={
        '<DATE>': 'Date',
        '<OPEN>': 'Open_h4',
        '<HIGH>': 'High_h4',
        '<LOW>': 'Low_h4',
        '<CLOSE>': 'Close_h4',
        '<TICKVOL>': 'TickVol',
        '<VOL>': 'Vol',
        '<SPREAD>': 'Spread'
    }, inplace=True)

    data_h4['Date'] = pd.to_datetime(data_h4['Date'], format='%Y.%m.%d')

    # Set Date column as the index
    data_h4.set_index('Date', inplace=True)

    data_h4 = data_h4[['Open_h4', 'High_h4', 'Low_h4', 'Close_h4']]

    return data_h4


def combine_ohlc_into_single_day(data_h4):
    grouped = data_h4.groupby(data_h4.index.date)

    # Create a new dataframe to store the result
    reshaped_h4 = pd.DataFrame()

    # Extract Open, High, Low, Close for each 4-hour period and reshape
    for date, group in grouped:
        group = group.reset_index(drop=True)
        for i in range(0, len(group)):
            if i == 0:
                reshaped_h4.at[date, f'Open_h4_{i}'] = group.loc[i, 'Open_h4']
                reshaped_h4.at[date, f'High_h4_{i}'] = group.loc[i, 'High_h4']
                reshaped_h4.at[date, f'Low_h4_{i}'] = group.loc[i, 'Low_h4']
                reshaped_h4.at[date, f'Close_h4_{i}'] = group.loc[i, 'Close_h4']
            else:
                reshaped_h4.at[date, f'Open_h4_{i}'] = group.loc[i, 'Open_h4']
                reshaped_h4.at[date, f'High_h4_{i}'] = group.loc[i, 'High_h4']
                reshaped_h4.at[date, f'Low_h4_{i}'] = group.loc[i, 'Low_h4']
                reshaped_h4.at[date, f'Close_h4_{i}'] = group.loc[i, 'Close_h4']

    return reshaped_h4
    

def add_ohlc_in_lagged(reshaped_h4, lag_by= 7):
    
    features_h4 = pd.DataFrame()
    for candles in range(0, 6): # 0 --> 5 all 6 candles
        for day in range(1, lag_by + 1): # last 3 days = 6 * 3 = Last H4 18 candles
 
            # new name will be candle number and day shifted from
            features_h4[f'Close_h4_{candles}_T-{day}'] = reshaped_h4[f'Close_h4_{candles}'].shift(day)
            features_h4[f'High_h4_{candles}_T-{day}']  = reshaped_h4[f'High_h4_{candles}'].shift(day)
            features_h4[f'Open_h4_{candles}_T-{day}']  = reshaped_h4[f'Open_h4_{candles}'].shift(day)
            features_h4[f'Low_h4_{candles}_T-{day}']  = reshaped_h4[f'Low_h4_{candles}'].shift(day)
    
    return  features_h4


def add_features(features_h4, lag_by= 7):
    
    # features_h4.fillna(0.0, inplace= True)
    features_h4 = features_h4.apply(lambda x: x.fillna(x.mean()), axis=0)
    
    for candles in range(0, 6): # 0 --> 5 all 6 candles
        for day in range(1, lag_by + 1): # last 3 days = 6 * 3 = Last H4 18 candles
            pass 
            # open_value  = features_h4[f'Open_h4_{candles}_T-{day}']
            close_value = features_h4[f'Close_h4_{candles}_T-{day}']
            high_value = features_h4[f'High_h4_{candles}_T-{day}']
            low_value  = features_h4[f'Low_h4_{candles}_T-{day}']
            hl2 = ( high_value + low_value ) / 2

            # features_h4[f'Low_h4_{candles}_T-{day}'].fillna(features_h4[f'Low_h4_{candles}_T-{day}'].mean(), inplace=True)
            # features_h4[f'High_h4_{candles}_T-{day}'].fillna(features_h4[f'High_h4_{candles}_T-{day}'].mean(), inplace=True)
            # features_h4[f'Close_h4_{candles}_T-{day}'].fillna(features_h4[f'Close_h4_{candles}_T-{day}'].mean(), inplace=True)
            features_h4[f'slow_harmonic_mean_{candles}_T-{day}'] = calculate_harmonic_mean(close_value , period=27)
            features_h4[f'fast_harmonic_mean_{candles}_T-{day}'] = calculate_harmonic_mean(close_value, period=9)
    
            features_h4[f'harmonic_mean_high_{candles}_T-{day}'] = calculate_hm_high(high_value)
            features_h4[f'harmonic_mean_low_{candles}_T-{day}']  = calculate_hm_low(low_value)
            
            rsi  = calculate_rsi(hl2, period=27)
            smoothed_rsi =  rsi.rolling(window=9).mean()
            slow_smoothed_rsi =  rsi.rolling(window=10).mean()

            features_h4[f'rsi_sma_fast_{candles}_T-{day}'] = smoothed_rsi
            features_h4[f'rsi_sma_slow_{candles}_T-{day}'] = slow_smoothed_rsi
            features_h4[f'rsi_{candles}_T-{day}'] =  rsi
            features_h4[f'rsi_crossover_fast_{candles}_T-{day}'] =  rsi_crossover(smoothed_rsi)
            features_h4[f'rsi_crossover_slow_{candles}_T-{day}'] =  rsi_crossover_with_sma(rsi , slow_smoothed_rsi)
            
            
            open_value  = features_h4[f'Open_h4_{candles}_T-{day}'].values
            close_value = features_h4[f'Close_h4_{candles}_T-{day}'].values
            high_value = features_h4[f'High_h4_{candles}_T-{day}'].values
            low_value  = features_h4[f'Low_h4_{candles}_T-{day}'].values

            open_val  = features_h4[f'Open_h4_{candles}_T-{day}']
            close_val = features_h4[f'Close_h4_{candles}_T-{day}']
            high_val = features_h4[f'High_h4_{candles}_T-{day}']
            low_val  = features_h4[f'Low_h4_{candles}_T-{day}']

            hlc = ( features_h4[f'High_h4_{candles}_T-{day}'] + features_h4[f'Low_h4_{candles}_T-{day}'] + features_h4[f'Close_h4_{candles}_T-{day}']) / 3 

            features_h4[f'true_range_h4_{candles}_T-{day}'] = pd.Series(high_value- low_value)

            # features_h4[f'median_h4_{candles}_T-{day}'] = pd.Series( (high_value + low_value) / 2 )
            
            
#             features_h4[f'RSI_slow_{candles}_T-{day}']   = calculate_rsi(close_val, period=21)
#             features_h4[f'RSI_fast_{candles}_T-{day}']   = calculate_rsi(close_val, period=9)
            
            features_h4[f'stdev_slow_{candles}_T-{day}'] = close_val.rolling(window=18).std()
            features_h4[f'stdev_fast_{candles}_T-{day}'] = close_val.rolling(window=9).std()
    
#             features_h4[f'Upper_Band_slow_{candles}_T-{day}'] = close_val.rolling(window=18).mean() + (close_val.rolling(window=5).std() * 2)
#             features_h4[f'Lower_Band_slow_{candles}_T-{day}'] = close_val.rolling(window=18).mean() - (close_val.rolling(window=5).std() * 2)

#             features_h4[f'Upper_Band_fast_{candles}_T-{day}'] = close_val.rolling(window=9).mean() + (close_val.rolling(window=5).std() * 2)
#             features_h4[f'Lower_Band_fast_{candles}_T-{day}'] = close_val.rolling(window=9).mean() - (close_val.rolling(window=5).std() * 2)

#             ema3_value  = hlc.ewm(span=3, adjust=False).mean()
#             ema5_value  = hlc.ewm(span=5, adjust=False).mean()
#             ema7_value  = hlc.ewm(span=7, adjust=False).mean()
#             ema14_value = hlc.ewm(span=14, adjust=False).mean()

            highest_high = features_h4[f'High_h4_{candles}_T-{day}'].rolling(window=9).max()
            lowest_low   = features_h4[f'Low_h4_{candles}_T-{day}'].rolling(window=9).min()

            # Calculate the relative range
            features_h4[f'relative_range_h4_{candles}_T-{day}'] = close_value - ( ( highest_high + lowest_low ) / 2)


            candle_type_value  = candle_type(open_value, high_value, low_value, close_value)


#             elastic_supertrend, es_status_value = ma_based_supertrend_indicator( high_value, low_value, close_value, atr_length=9, atr_multiplier=2.5, ma_length=9)

#             elastic_supertrend_crossover = supertrend_status_crossover(es_status_value)

            supertrend, supertrend_status_value = supertrend_indicator(high_value, low_value, close_value, period= 27, multiplier=5.5)
            supertrend_crossover = supertrend_status_crossover(supertrend_status_value)

            features_h4[f'supertrend_h4_{candles}_T-{day}'] = supertrend

            features_h4[f'supertrend_status_h4_{candles}_T-{day}'] = supertrend_status_value

            features_h4[f'supertrend_crossover_h4_{candles}_T-{day}'] = supertrend_crossover


#             features_h4[f'es_supertrend_h4_{candles}_T-{day}'] = elastic_supertrend

#             features_h4[f'es_supertrend_crossover_h4_{candles}_T-{day}'] = elastic_supertrend_crossover

#             features_h4[f'es_supertrend_status_h4_{candles}_T-{day}'] = es_status_value

            
            features_h4[f'candle_type_h4_{candles}_T-{day}'] = candle_type_value

            # features_h4[f'smi_direct_h4_{candles}_T-{day}']  = smi_fast_direction

            # features_h4[f'smi_h4_{candles}_T-{day}'] = smi_fast

#             features_h4[f'ema3_h4_{candles}_T-{day}']  = pd.Series(ema3_value)

#             features_h4[f'ema5_h4_{candles}_T-{day}']  = pd.Series(ema5_value)

#             features_h4[f'ema7_h4_{candles}_T-{day}']  = pd.Series(ema7_value)

#             features_h4[f'ema14_h4_{candles}_T-{day}'] = pd.Series(ema14_value)

            ha_open, ha_close, ha_high, ha_low = heikin_ashi_candles(open_value, high_value, low_value, close_value)
            heikin_ashi_candle = heikin_ashi_status(ha_open, ha_close)
            features_h4[f'heikin_ashi_{candles}_T-{day}'] = heikin_ashi_candle

#             selected_columns = features_h4[[ema3_column, ema5_column, ema7_column ]]

#             ema_mean = np.mean(selected_columns, axis=1)
#             features_h4[f'ema_difference_{candles}_T-{day}'] = pd.Series(np.subtract( ema_mean, ema14_value ))

        
    return  features_h4



def ema( price, period):

  price = np.array(price)
  alpha = 2 / (period + 1.0)
  alpha_reverse = 1 - alpha
  data_length = len(price)

  power_factors = alpha_reverse ** (np.arange(data_length + 1))
  initial_offset = price[0] * power_factors[1:]

  scale_factors = 1 / power_factors[:-1]

  weight_factor = alpha * alpha_reverse ** (data_length - 1)

  weighted_price_data = price * weight_factor * scale_factors
  cumulative_sums = weighted_price_data.cumsum()
  ema_values = initial_offset + cumulative_sums * scale_factors[::-1]

  return ema_values
    


# Calculate RSI
def calculate_rsi(series, period=5):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

    
def moving_max(array, window_size):
   
    rolling_max = np.full(array.shape, 0.0)
    
    for i in range(len(array) - window_size + 1):
        window_values = array[i:i + window_size]
        rolling_max[i + window_size - 1] = np.max(window_values)
        
    rolling_max[np.isnan(rolling_max)] = np.nanmean(rolling_max)
    return rolling_max    
    

def moving_min(array, window_size):

    rolling_min = np.full(array.shape, 0.0)
    for i in range(len(array) - window_size + 1):
        window_values = array[i:i + window_size]
        rolling_min[i + window_size - 1] = np.min(window_values)
        
    rolling_min[np.isnan(rolling_min)] = np.nanmean(rolling_min)
    return rolling_min



def true_range( high, low, close):

  close_shift = shift(close, 1)
  high_low, high_close, low_close = np.array(high - low, dtype=np.float32), \
                                    np.array(abs(high - close_shift), dtype=np.float32), \
                                    np.array(abs(low - close_shift), dtype=np.float32)

  true_range = np.max(np.hstack((high_low, high_close, low_close)).reshape(-1, 3), axis=1)

  return true_range


def shift(array, place):

  array = np.array(array, dtype=np.float32)
  shifted = np.roll(array, place)
  shifted[0:place] = np.nan
  shifted[np.isnan(shifted)] = np.nanmean(shifted)

  return shifted


def ma_based_supertrend_indicator( high, low, close, atr_length=10, atr_multiplier=3, ma_length=10):

    # Calculate True Range and Smoothed ATR
    tr = true_range(high, low, close)
    atr = ema(tr, atr_length)

    upper_band = (high + low) / 2 + (atr_multiplier * atr)
    lower_band = (high + low) / 2 - (atr_multiplier * atr)

    trend = np.zeros(len(atr))

    # Calculate Moving Average
    ema_values = ema(close, ma_length)

    if ema_values[0] > lower_band[0]:
        trend[0] = lower_band[0]
    elif ema_values[0] < upper_band[0]:
        trend[0] = upper_band[0]
    else:
        trend[0] = upper_band[0]

    # Compute final upper and lower bands
    for i in range(1, len(close)):
        if ema_values[i] > trend[i - 1]:
            trend[i] = max(trend[i - 1], lower_band[i])


        elif ema_values[i] < trend[i - 1]:
            trend[i] = min(trend[i - 1], upper_band[i])

        else:
            trend[i] = trend[i - 1]

    status_value = np.where(ema_values > trend, 1.0, -1.0)

    return trend, status_value




def supertrend_status_crossover( status_value):


    prev_status = np.roll(status_value, 1)
    supertrend_status_crossover = np.where((prev_status < 0) & (status_value > 0), 1.0, np.where((prev_status > 0) & (status_value < 0), -1.0, 0))

    return supertrend_status_crossover




def supertrend_indicator(high, low, close, period, multiplier=1.0):

    true_range_value = true_range(high, low, close)

    smoothed_atr = ema(true_range_value, period)

    upper_band = (high + low) / 2 + (multiplier * smoothed_atr)
    lower_band = (high + low) / 2 - (multiplier * smoothed_atr)

    supertrend = np.zeros(len(true_range_value))
    trend = np.zeros(len(true_range_value))

    if close[0] > upper_band[0]: trend[0] = upper_band[0]
    elif close[0] < lower_band[0]: trend[0] = lower_band[0]
    else:  trend[0] = upper_band[0]

    for i in range(1, len(close)):

        if close[i] > upper_band[i]: trend[i] = upper_band[i]
        elif close[i] < lower_band[i]: trend[i] = lower_band[i]
        else: trend[i] = trend[i - 1]

    # Calculate Buy/Sell Signals using numpy where  # np.where( close > trend, '1 Buy', '-1 Sell')
    status_value = np.where(close > trend, 1.0, -1.0)

    return trend, status_value

def supertrend_status_crossover(status_value):


    prev_status = np.roll(status_value, 1)
    supertrend_status_crossover = np.where((prev_status < 0) & (status_value > 0), 1.0, np.where((prev_status > 0) & (status_value < 0), -1.0, 0))

    return supertrend_status_crossover




def direction_crossover_signal_line(signal, signal_ema):

    direction = np.where(signal - signal_ema > 0, 1, -1)
    prev_direction = np.roll(direction, 1)
    crossover = np.where((prev_direction < 0) & (direction > 0), 1,
                          np.where((prev_direction > 0) & (direction < 0), -1, 0))

    return direction, crossover


def stochastic_momentum_index(high, low, close, period=20, ema_period=5):
    # Compute highest high and lowest low over the period
    highest_high = high.rolling(window=period).max()
    lowest_low = low.rolling(window=period).min()

    # Compute relative range
    relative_range = close - ((highest_high + lowest_low) / 2)

    # Compute highest-lowest range
    highest_lowest_range = highest_high - lowest_low

    # Smooth relative range and highest-lowest range
    relative_range_smoothed = relative_range.ewm(span=ema_period, adjust=False).mean().ewm(span=ema_period, adjust=False).mean()
    highest_lowest_range_smoothed = highest_lowest_range.ewm(span=ema_period, adjust=False).mean().ewm(span=ema_period, adjust=False).mean()

    # Calculate SMI
    smi = (relative_range_smoothed / (highest_lowest_range_smoothed / 2)) * 100
    smi[smi == np.inf] = 0  # Replace infinite values with 0
    smi_ema = smi.ewm(span=ema_period, adjust=False).mean()

    return smi, smi_ema

# def stochastic_momentum_index(high, low, close, period=20, ema_period=5):

#     lengthD = ema_period
#     lowest_low   = moving_min(low, period)
#     highest_high = moving_max(high, period)
#     relative_range = close - ((highest_high + lowest_low) / 2)
#     highest_lowest_range = highest_high - lowest_low

#     relative_range_smoothed = ema(ema(relative_range, ema_period), ema_period)
#     highest_lowest_range_smoothed = ema(ema(highest_lowest_range, ema_period), ema_period)

#     smi = [(relative_range_smoothed[i] / (highest_lowest_range_smoothed[i] / 2)) * 100 if
#             highest_lowest_range_smoothed[i] != 0 else 0.0
#             for i in range(len(relative_range_smoothed))]

#     smi_ema = ema(smi, ema_period)

#     return smi, smi_ema


def candle_type(o, h, l, c):

    diff = abs(c - o)
    o1, c1 = np.roll(o, 1), np.roll(c, 1)  #
    min_oc = np.where(o < c, o, c)
    max_oc = np.where(o > c, o, c)

    pattern = np.where(
      np.logical_and( min_oc - l > diff, h - max_oc < diff), 6,
      np.where(np.logical_and( h - max_oc > diff, min_oc - l < diff),
      4, np.where(np.logical_and(np.logical_and(c > o, c1 < o1), np.logical_and(c > o1, o < c1)),
        5, np.where( min_oc - l > diff, 3,
                      np.where(np.logical_and( h - max_oc > diff,
                  min_oc - l < diff),
                      2, np.where(np.logical_and(np.logical_and(c > o, c1 < o1), np.logical_and(c > o1, o < c1)),
                      1, 0))))))
    return pattern





def heikin_ashi_status( ha_open, ha_close):

    candles = np.full_like(ha_close, '', dtype='U10')

    for i in range(1, len(ha_close)):

        if ha_close[i] > ha_open[i]: candles[i] = 2 #'Green'

        elif ha_close[i] < ha_open[i]: candles[i] = 1 # 'Red'

        else: candles[i] = 0 #'Neutral'

    return candles

def heikin_ashi_candles( open, high, low, close):

    ha_low, ha_close = np.empty(len(close), dtype=np.float32), np.empty(len(close), dtype=np.float32)
    ha_open, ha_high = np.empty(len(close), dtype=np.float32), np.empty(len(close), dtype=np.float32)

    ha_open[0] = (open[0] + close[0]) / 2
    ha_close[0] = (close[0] + open[0] + high[0] + low[0]) / 4

    for i in range(1, len(close)):
        ha_open[i] = (ha_open[i - 1] + ha_close[i - 1]) / 2
        ha_close[i] = (open[i] + high[i] + low[i] + close[i]) / 4
        ha_high[i] = max(high[i], ha_open[i], ha_close[i])
        ha_low[i] = min(low[i], ha_open[i], ha_close[i])

    return ha_open, ha_close, ha_high, ha_low
















